# Anupreet Singh's GA1 Implementation(Campus ID-Uk43298)
Disclaimer: 
* The markdown sections that say Tech Note are notes for myself as the developer. They don't give description in regards to structure of the Jupyter Notebook.
* Any mardown section that doesn't say Tech Note/TN is intended for the perusal of the reviewer. 

In [ ]:
## run these 2 commands in a terminal to activate a virtual environment to be used for the purpose of this .ipynb file
# cd '/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation'
# source GA1env/bin/activate
# then close file and open again


##### Tech Note(Most Basic things about Coding):
Statement are the most basic intructions that the interpreter executes. For Example: Assignment(x=6), control flow statements(if, for, while, with), import statement(import torch), function definition(def fun).  

Two types of Parameters in Function Definition:-
1. Requried Parameters are the Uninstantiated variables x and y in def fun(x, y, age="20", year="2021"), required parameters are often placed first in function definitions
2. Optional Parameters are the instantiated variables age and year in def fun(x, y, age="20", year="2021"), these have a default value already in the function definition so it isnt necessary to pass arguments for them. They usually come after required parameters

Two possible ways of passing arguments in a function:-
1. You could either just send in argument as raw values or some variable name in the order that the parameters are set in the function definition.
2. You could use the keyword argument irrespective of the order, where the name of parameter from the function definition is used to instantiate it to a value of choice in the function call.

## Section 1-Data Preparation and Fine Tuning the model

##### TN(Pip Package):
When I install a certain pip package it remains installed in my virtual environment directory even if I close my vs code entirely, but when I open the .ipynb file and select the kernel that points to the python interpreter in that virtual environment then I can import those package in the current runtime of my kernel by using import.

In [ ]:
## Installing Required Libraries
%pip install transformers
%pip install "numpy<2" #this installs numpy version 1. and is crucial as numpy version 2.0(which is automatically installed when installing transformers) does not work well with torch
%pip install datasets
%pip install torch
%pip install 'accelerate>=0.26.0' #to be used for Trainer
%pip install sacrebleu

#These are needed only in jupyter notebook running locally to display things like progress bars in output cells
%pip install ipywidgets
%pip install sentencepiece
%pip install tqdm #for showing progress bars


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.3 MB/s eta 0:00:00
  

In [69]:
## Importing Libraries
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, MarianMTModel, MarianTokenizer
from transformers import Trainer, TrainingArguments
import numpy as np
import torch
import torch.nn.functional as F #is a module that provides several functions for pytorch tensors commonly used in neural networks related tasks
from datasets import load_dataset, load_from_disk
import os #to make changes to files in local disk
import accelerate
from tqdm import tqdm
import sacrebleu

In [70]:
## Downloading the datasets

#Uncomment one of the below to download wmt-16 or 19
dataset = load_dataset('wmt16', 'ru-en') #This downloads the smaller dataset into our working environment
#dataset.save_to_disk('/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/WMT-16(en-ru)')
#TN- .save_to_disk huggingface function already included in the dataset object(created uing load_dataset function) that saves processed dataset to our local storage, it always saves in .arrow files.

#Uncomment one of the below to download wmt-19
#dataset = load_dataset('wmt19', 'ru-en') #This downloads the bigger dataset into our working environment
#dataset.save_to_disk('/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/WMT-19(en-ru)')

In [ ]:
## Loading dataset from local storage to our kernel
dataset=load_from_disk('/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/WMT-16(en-ru)')

##### TN(Dataset Structure):
1. For hugginggace datasets When you print dataset["train"] what you see is not the actual arrangement of the train split of dataset but a high level summary of the datasets stucture, since the actual dataset is too big and it would be asinine to print it anyway.

2. What is happening in the code in the second cell below is that you go to train split of the dataset, it has a column(the only feature as seen in the metadata of stucture). In that column you select which row you want to go to, each row has a dictionary(in this case key value pair of the 2 languages), and then you access the english part of the language.

In [ ]:
# Seeing the structure of the dataset
print(dataset["validation"])

for i in range(5):
    print(dataset["train"]["translation"][i]["en"])

Dataset({
    features: ['translation'],
    num_rows: 2818
})


##### TN(Translation Using mT5):
The two methods to generate an english to russian translation using mT5 are:
1. Use a prefix in the english sentence indicating that task(which turns out performs extremely poorly so the second method is the basically the only option to get meaningful results)
2. Fine tune the model on a large dataset containing pair of english and russian sequences so that it implicitly learns what to do.

In [ ]:
## Loading the mT5 tokenizer and model
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-base")#this downloads the tokenizer into our local working environment
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
## moving the model to the correct device
device=torch.device('mps') #moving the model to integrated GPU on macOS, used "cuda" for dedicated GPU and "cpu" for cpu
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62518, 512, padding_idx=62517)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62518, 512, padding_idx=62517)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

##### TN (Tokenizer)
In the Arguments of Tokenizer:
* "max_length"=128 and "truncation"=True means that any sequence in the batch(of size 1000 by default) longer than will be truncated to 128
* "padding"=True means that all sequences will be padded to the lenght of the longest sequence in the batch.
* "padding"=max_length means that all sequences in the batch will be padded to the length of 128 in this case.


In the Output of the tokenizer we get a dictionary containing two key-value pairs:
* "input_ids" are the numerical representation of tokens in form of pytorch tensors. Pytorch tensors could be visualized as 2D matrix where each row correspond to token ID of one sentence and each column correspond to token ID in the sentence
* "attention mask" tells which tokens are paded tokens so they can be ignored during training

In [ ]:
##Preprocessing/tokenizing the WMT dataset to generate vocabulary
def preprocess_function(examples):
    source_tokens = tokenizer([example['en'] for example in examples['translation']],
                        return_tensors='pt',
                        padding="max_length", #this is done because the Trainer() expects all the sequence in its training batch to be of the same length, so we make all sequence of the same lenght since training batch cant be made as big as 1000 due to computation constraints and accuracy concerns.
                        truncation=True,
                        max_length=128)# altough in the train split the longest english and russian sequences are 5194 and 4886 words long, but sinces we just need to compare two performance for now we will use max_length=128, we could increase this to 256 or 512 but that would just require extra memory
    target_tokens = tokenizer([example['ru'] for example in examples['translation']],
                        return_tensors='pt',
                        padding="max_length",
                        truncation=True,
                        max_length=128)
    source_tokens['labels'] = target_tokens['input_ids']#Here we create a key-labels and store input_ids of target language as value in it to correspond with input_ids of source language,which will be used for model training
    # source_tokens['labels_attention_mask'] = target_tokens['attention_mask']#we could also include target attention masks if needed

    return source_tokens#and then we just return the source_tokens as it has both language input_ids too


# Process and save immediately
#TN- The .map() function processes the dataset in batches of size 1000 by default. During this, intermediate results (like tokenized data) are cached in our local directory to avoid redoing the computations in the future.
train_T_dataset = dataset['train'].map(
    preprocess_function,
    batched=True,
)
# Removing the 'translation' column from the tokenized dataset and then saving it to local disk
train_T_dataset = train_T_dataset.remove_columns(['translation'])
train_T_dataset.save_to_disk('/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/WMT-16(en-ru) train_T')


eval_T_dataset = dataset['validation'].map(
    preprocess_function,
    batched=True,
)
# Removing the 'translation' column from the tokenized dataset and then saving it to local disk
train_T_dataset = eval_T_dataset.remove_columns(['translation'])
eval_T_dataset.save_to_disk('/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/WMT-16(en-ru) eval_T')




#You could use the line below as an argument in .map to remove the column from the dataset loaded in kernel to free up some memory, but it is not gonna do much here
#remove_columns=dataset['train'].column_names
#remove_columns removes the provided column name(in this case just one column named translation,which is basically the entire dataset) from kernel memory



Map:   0%|          | 0/1516162 [00:00<?, ? examples/s]

Saving the dataset (0/6 shards):   0%|          | 0/1516162 [00:00<?, ? examples/s]

Map:   0%|          | 0/2818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2818 [00:00<?, ? examples/s]

In [ ]:
## Deleting the cache on the local diskc created during preprocessing
#Since we saved the processed dataset to disk we can now delete the cache files which have been created in our original dataset folder when it was being tokenized by the .map function
# Define the path to your dataset directory
train_dir = '/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/WMT-16(en-ru)/train'
eval_dir = '/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/WMT-16(en-ru)/validation'
# Function to delete cache files
def delete_cache_files(directory):
    # Loop through files in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        # Check if the file contains "cache" in its name and ends with .arrow
        if 'cache' in filename and filename.endswith(".arrow"):
            print(f"Deleting cache file: {file_path}")
            os.remove(file_path)  # Delete the file

# Call the function to delete cache after saving
delete_cache_files(train_dir)
delete_cache_files(eval_dir)



Deleting cache file: /Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/WMT-16(en-ru)/train/cache-7af736c62e1f0b78.arrow
Deleting cache file: /Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/WMT-16(en-ru)/validation/cache-5122a4982e8ec6c1.arrow


In [ ]:
##loading the Tokenized dataset from local disk
train_T_dataset=load_from_disk('/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/WMT-16(en-ru) train_T')
eval_T_dataset=load_from_disk('/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/WMT-16(en-ru) eval_T')

##### TN(Tokenized dataset):
1. Each feature of the dataset is actually a column, and the tokenized dataset here contains the original translation column from the train split as well as 4 other columns that we put into it using the tokenizer.
2. We removed the "translation" column and dont add the "labels_attention_mask" column for now

##### TN (Trainer):
1. The batch size tells that in 1 step how many examples does the optimizer go over before updating the models parameters

2. By default the Trainer class uses the AdamW optimizer for adjusting the weights of the transformer models like mT5. AdamW optimizer can be thought of as an optimization algorithm like gradient descent but a much better version of it.

3. weight_decay works by penalizing larger weights to avoid overfitting to the training set. This is because if the model has learned very large weights for a particular feature then the predictions may be dominated by one or more such features, causing the model to overfit to the peculiarities of the training data.

In [ ]:
## Fine tuning the model to the dataset
training_args = TrainingArguments(
    output_dir='/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/Model/model training checkpoints', # output directory where the model predictions and checkpoints will be saved, checkpoints are state of the model at that time
    save_steps=10_000,               # save checkpoints after every 10,000 steps in the output_dir
    save_total_limit=2,              # limit the number of checkpoints to save
    logging_dir='/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/Model/Model training log', # directory for storing logs
    logging_steps=500,               # log training metrics(like loss) at every 500 steps in the logging_dir
    evaluation_strategy="epoch",     # by this strategy we evaluate the model after every epoch
    learning_rate=5e-5,              # learning rate for the optimizer
    per_device_train_batch_size=10,   # batch size for training
    per_device_eval_batch_size=4,    # batch size for evaluation
    num_train_epochs=3,              # number of training epochs
    weight_decay=0.01,               # strength of weight decay
    no_cuda=True,                    #This automatically sets model.device=cpu, I am Using the CPU for tuning the model, since my GPU is not good
    dataloader_num_workers=4,        # Use 4 CPU cores for data loading
)

# Initialize the Trainer, the Trainer class from huggingface handles fine tuning the model(AKA adjusting the models parameters according to the dataset to minimize loss)
trainer = Trainer(
    model=model,                       # the model to train
    args=training_args,                # training arguments
    train_dataset=train_T_dataset,   # the tokenized dataset for training
    eval_dataset=eval_T_dataset.select(range(10)),    # the tokenized dataset for evaluation (same dataset for simplicity)
    tokenizer=tokenizer
)

# first_100_rows = train_T_dataset.select(range(100)) , select()could be used to select the required rows from the dataset


# Start the fine-tuning process
trainer.train()

# Save the fine-tuned model
model.save_pretrained('/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/Model/Fine-tuned Model')
tokenizer.save_pretrained('/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/Model/Fine tuned Tokenizer')


/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/GA1env/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/GA1env/lib/python3.12/site-packages/transformers/training_args.py:1583: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
/var/folders/mg/wpj49_8s7yb8s_vtb6b2x7xc0000gn/T/ipykernel_28147/2279088603.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/454851 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
## Loading the fine tuned model
# Load the model from local disk
model = MT5ForConditionalGeneration.from_pretrained('/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/Model/Fine-tuned Model')

# Load the tokenizer from local disk
tokenizer = MT5Tokenizer.from_pretrained('/Users/manpreetsingh/Downloads/UMBC/NLP 673/Graduate Assesment/GA1 Implementation/Datasets/Model/Fine tuned Tokenizer')

## Section 2: Using the Model's .generate function to translate the eval split

##### TN(Translating using .generate):
1. Putting the model in evaluation mode is necessary because it then dropout is disabled, so no neurons are randomly dropped and the whole network is used for making predicitions.

2. The models .generate function expects all tokenized sequences to be of the same length, which is why attention mask is used to tell the model which token ids are padded in a sequence so it could ignore it. This is why we pad all sequneces to length of longest sentence in the eval dataset by padding=True.

3. When calculating BLEU score using sacrebleu.corpus(hypothesis, references).The function expects first argument "hypothesis" to be a list of string sequences generated by the model and second argument "reference" to be list(Outer) of lists(Inner) of reference translations where each list(inner) is one set of reference string sequences.This is so that each hypothesis sequences could possibly be compared to more than 1 possible reference translation.

4. Another Big Thing I found about BLEU score(through falling in a valley of errors) is that it does not raise an error even if their is signicant mismatch between the number of hypotheses and reference sequences so always check length of the list containing sequences before calculating BLEU score.

In [ ]:
## Loading Helinski Tokenizer and Model
# Loading the tokenizer
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")#this downloads the tokenizer into our local working environment

# Loading the pre-trained model for conditional generation (translation)
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-ru")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [5]:
## moving the model to the correct device
device=torch.device('cuda') #moving the model to integrated GPU on macOS, used "cuda" for dedicated GPU and "cpu" for cpu
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62518, 512, padding_idx=62517)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62518, 512, padding_idx=62517)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [ ]:
## Using .generate to get translation of "validation" split of dataset
eval_dataset=dataset["validation"]
generation_batch_size=30 #increase or decrease as per memory available in models processing unit
model.eval()# Putting the model in eval mode
def translate(dataset):
    #tokenize the dataset at once because it is a small dataset and tokenization does not need to be done in batches
    dataset_t = tokenizer([row["en"] for row in dataset["translation"]],
                           return_tensors="pt",
                           padding=True,
                           truncation=True,
                           max_length=128).to(device) #we move the tensors to the device that our model is on

    # Generate translation for batches of tokenized dataset and add them to translated_dataset list, we do this in batches to speed up the process by making use of the parallel procesing but we dont do it all at once because of possible memory constraints.
    input_ids=dataset_t["input_ids"] #This will be a 2D tensor of shape(total number of sequences tokenized, i.e. lenght of list passed in tokenizer, sequence length)
    attention_mask=dataset_t["attention_mask"] #This will be 2D tensor of shape()
    translated_dataset=[] #The list of generated translation which will be returned from the function
    for i in tqdm(range(0,input_ids.shape[0], generation_batch_size)):
        batch_input_ids= input_ids[i:i+generation_batch_size] #slicing for tensors works in the same way as slicing for lists, so if i+generation_batch_size>input_ids.shape[0] then slicing will just return a tensor until the last element of the parent tensor.
        batch_attention_mask=attention_mask[i:i+generation_batch_size]
        outputs = model.generate(input_ids=batch_input_ids, attention_mask=batch_attention_mask, num_beams=4, early_stopping=True)
        translated_batch=tokenizer.batch_decode(outputs, skip_special_tokens=True) # tokenizer.decode could also handle 2D tensor containing muliple token_ids but .batch_decode is said to handle it faster
        translated_dataset.extend(translated_batch) # .extend adds each element from an iterable to a list, whereas if we used .append here it would have added the entire iterable to the list as a single element

    return translated_dataset


# Translate all EN sentences
model_generated_list = translate(eval_dataset)
reference_list=[[row["ru"] for row in eval_dataset["translation"]]] #This creates a list containing one list of string sequences, a list containing one set of reference translations

#BLEU score for model generation
bleu_score = sacrebleu.corpus_bleu(model_generated_list, reference_list)
print(f"BLEU score for validation split using .generate:{bleu_score}")





100%|██████████| 94/94 [40:05<00:00, 25.60s/it]   


BLEU score for validation split using .generate:BLEU = 27.11 56.9/32.9/21.0/13.8 (BP = 1.000 ratio = 1.027 hyp_len = 57419 ref_len = 55920)


##### TN(Translating using custom function "scratch_beam_search"):
1. As per the requirements of the task, we just replace .generate function from the cell above with our custom function that behaves exactly the same.
2. By inspection of tokenizer.special_tokens_map and model.config for the helsinki model we find out that:
    * eos_token_id=0
    * unk_token_ids=1
    * pad_token_id=62517
    * decoder_start_token_id=62517, so the model is using the same token id to padd sequences and as a starting token
3. For the tokenizer, the encoder converts text into token_ids and the decoder converts token_ids back into human readable text

4. For the Model, the encoder process the input_ids of the source sequence and produces a hidden representation(context vector) that summarizes its meaning and the decoder takes encoder hidden representation and decoder_inputs_ids(output tokens generated upto that point) and produces logits corresponding to each next token for every sequence of decoder_input_ids. This process of generating logits once is called a Forward Pass.


## Section 3: Using the Model's forward pass in custom beam search function to translate the Eval split

In [6]:
## Using Custom function "scratch_beam_search" in place of .generate() to Translate Validation split of dataset
eval_dataset=dataset["validation"]
reference_list_BLEU=[[row["ru"] for row in eval_dataset["translation"]]] #This creates a list containing one list of string sequences, a list containing one set of reference translations
generation_batch_size=1 #increase or decrease as per memory available in models processing unit
model.eval()# Putting the model in eval mode

def scratch_beam_search(input_ids, attention_mask, num_beams=4, max_length=128, early_stopping=True):

    original_batch_size = input_ids.shape[0] #This  original_batch_size will be the number of sequences sent into scratch_beam_search function
    """
    This code below was causing the problem of not being able to use genertaion_batch_size>1 cause we were using .repeat function when we should have used repeat_interleave() function
    The problem was because it would cause misalignment with decoder_input_ids when they are sent in the model during Forward pass

    # Create a copy of input_ids and attention masks of each sequence for each beam
    input_ids = input_ids.repeat(num_beams, 1) #.repeat is a pytorch function that creates a tensor by repeating the entire tensor along a its dimensions the specified number of times(number of copies along dim=0(row), number of copies of along dim=1(column)), here it creates "num_beams" copies of input_ids along the 1st dimension(batch_size) of the 2D tensor, and keeps everything along 2nd dimension(sequence length) unchanged since we pass in the 1
    attention_mask = attention_mask.repeat(num_beams, 1)  #since we are creating as much copies as num_beams you could see why more beams would result in more memory usage and computational overhead

    """
    input_ids = input_ids.repeat_interleave(num_beams, dim=0) #.repeat_interleave() repeats each element of along num_beam number of time along the specified dimension
    attention_mask = attention_mask.repeat_interleave(num_beams, dim=0) #since we are creating as much copies as num_beams you could see why more beams would result in more memory usage and computational overhead


    # Creating a 2D-tensor with one "beginning of sentence" token_id for each possible output sequence(input sequences*number of beams)
    decoder_input_ids = torch.full(
                        (original_batch_size * num_beams, 1), #torch.full allows us to create a tensor of shape=(batch_size * num_beams, 1) and fill each of its elements with value=model.config.decoder_start_token_id(62517 in case of helsinki)
                        model.config.decoder_start_token_id,
                        dtype=torch.long, #an optional argument to specify the data type of each element, torch.long is suitable for storing 64 bit signed integers
                        device=device # an optional argument to specify which device to store the tensor on, This is a fresh tensor so it best to store it on the device that our model is on
                        )
    #batch_size of inputs_ids, attention masks, and decoder_input_ids is orginal_batch_size time num_beams.

    # Creating a 1D tensor for storing total scores for each sequence upto current step in decoder_input_ids
    beam_scores = torch.zeros(              #torch.zeros creates a tensor with all elements set to zeros
                    original_batch_size * num_beams, #since we only pass one value in shape, it creates a 1D tensor
                    dtype=torch.float, #torch.float refers to torch.float32 used for storing 32 bit floating point numbers
                    device=device
                    )
    #finished_beams = torch.zeros(original_batch_size * num_beams, dtype=torch.bool, device=device) #keeping track of which beams have generated EOS tokens and finished generation
    Start=True ##initial Condition to be checked only once during the first forward pass
    # Generate sequences
    for step in range(max_length):

        with torch.no_grad(): #with statement allows us to use context manager(CM) to ensure proper setup(_enter_ method of CM) and cleanup(_exit_ method of CM) and manage resources effectively. torch.no_grad in a CM that disables gradient computation since it is an unnecessary computation overhead during inference.

            # Forward pass
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                decoder_input_ids=decoder_input_ids
                ) #This will be a dictionary like object containing a lot of things like hidden states,etc, but we are most interested in logits
            """
            Talking about some important Tensor Operations:

            Broadcasting:
            In pytorch, a process called broadcasting happens just before an element wise operation that requires two tensors with same number of dimensions to align in shape.
            Example Adding tensor A of shape(4,6,8) with tensor B of shape(4,1,1) then the scaler values in tensor B along the dimensions with only 1 element are duplicated to match the shape of the tensor A.
            But broadcasting only happens when two tensors have same size of some dimensions and some dimensions of size 1.

            tensor.view(size of dim=0, size of dim=1.....) is used to create a reshaped tensor with the same number of elements as the original tensor.
            It can add new dimensions of a specific size as long as the number of element in the original tensor could be accomodated.
            tensor.view(-1,10) tell us to accomodate 10 elements in dim=10 and infer what number of element to accomodate in dim=0 such that the total number of elements remain the same.

            tensor.unsqueeze() is used to add a singleton dimension(dimension of size 1) at a specified index
            tensor.unsequeeze(1) adds a singleton dimension at index=1

            tensor.squeeze() is used remove a singleton dimension from a specified index

            tensor.expand() is used to basically perform broadcasting on command, you would send in sizes of dimension of tensor which are capable of broadcasting and get a result.
            This means that the size you send in should either be the same as sizes of dimension of original tensor OR in case of singleton dimension in original tensor you could send in a greater number to stretch along that dimension
            tensor.expand(-1, 4) tells to keep dim=0 as is and basically stretch dim=1 to size 4, this is conditioned on the fact that the dimension to be stretched is of size 1 otherwise an error will occur

            tensor.expand_as(target_tensor) is used to perform broadcasting on tensor conditioned on the fact that the size of target_tensor is suitable for broadcasting


            """
            # Creating a 2D tensor Logits
            logits = outputs.logits[:, -1, :] #output.logits is a 3-D tensor of shape(original_batch_size*num_beam, current sequence length in decoder_input_ids, vocab size). where each element is the logit of each token_id of each sequence for each element of the vocabulary
                                              #By slicing we choose the last token_id of each sequence it to get a 2-D tensor of shape(original_batch_size*num_beam, vocab_size) where each element is a logit of last token_id of each sequence element for each element of the vocabulary
            #print(f"Logits at step {step}:\n{logits}")
            # Using Boolean masking to mark finished beams (those that already generated EOS)
            #finished_mask = finished_beams.view(-1, 1).expand(-1, logits.size(-1))  # finished_mask will be a 2D of shape(original_batch_size * num_beams, vocab_size) with values of True/False repeated in each row(representing each beam) upto the vocab_size
            #logits[finished_mask] = -float('inf')  # Making logits of lasts token for all elements of vocabulary for the finished beams negative infinity, By sending in boolean mask tensor of the same shape as the logits tensor the operation is applied to all logits elements that correspond with True in finished mask.
                                                    #Making Logit for all words in vocabulary negative infinity means probability=0 after softmax, and log of that is a big negative value, when that big negative value is combined with beam_score to get next_score it ensures that these beams will now not come in the topk.
                                                    #This ensures that only active beams (those not yet finished) are considered during the top-k selection.

            # Creating 2D tensor log probabilities
            log_probs = F.log_softmax(logits, dim=-1) #performs softmax on all logits normalizing them to probability of 0-1, then takes log of that so the highest probability is turend into the largest number(smallest negative value).
                                                      #dim=-1 takes softmax along the last dimension(2nd dimension with size=vocab_size in this case) which means that normalization happens for each row(last token for each sequence of decoder_input_ids)
            #print(f"Log_probs at step {step}:\n{log_probs}")
            #get top num_beams scores from each beam
            total_top_log_probs, total_top_token_ids=torch.topk(log_probs, num_beams, dim=1) #give 2 tensors of shape(original_batch_size*num_beams, num_beams) where each row contains scores top num_beam scores for that row and the corresponding token_ids in descending
            #print(f"Topk log_probs at step {step}:\n{total_top_log_probs}")
            #print(f"Topk token_ids at step {step}:\n{total_top_token_ids}")

            if Start==True:
                new_tokens=total_top_token_ids[::num_beams].reshape(-1,1) #using slicing with step value num_beam we take the top 4 tokens of a beam of a sequence and make a tensor containing new tokens for each beam, .reshape is used in place of .view() becauce we used slicing and so tensor is not contiguous in memory hence .view wont work
                decoder_input_ids=torch.cat([decoder_input_ids[torch.arange(original_batch_size*num_beams)],new_tokens], dim=1) #adding first 4 tokens to different beam of each sequence
                beam_scores=total_top_log_probs[::num_beams].reshape(-1) #slicing to make tensor of shape(original_batch_size,num_beams) and then we flatten it using .reshape(-1)
                Start=False
                #print(f"Decoder_input_ids at step{step}:\n{decoder_input_ids}")
                #print(f"Beam score at step{step}:\n{beam_scores}")

                continue

            #calculate total scores for each beams top 4 tokens
            total_top_score=total_top_log_probs+beam_scores.unsqueeze(1)
            #print(f"Total score of each topk token of each beam scores at step {step}:\n{total_top_score}")


            #reshape that
            reshaped_total_top_score=total_top_score.view(original_batch_size, num_beams*num_beams)

            #Getting the best scores for this step in generation
            new_top_scores,new_token_indices=torch.topk(reshaped_total_top_score,num_beams, dim=1) #gives 2 tensors of shape (original_batch_size, num_beam) containing the top num_beam score from all total_scores of one sequence in each row in descending order and their index in that row in total_top_score_reshaped
            #print(f"Total Top scores for new tokens of each sequence scores at step {step}:\n{new_top_scores}")
            #calculating beam indices for the new tokens of each sequence with best score
            beam_indices=new_token_indices//num_beams #elements of a row in this tensor will tell which token originally belonged to which beam for a sequence
            #reshaping beam_indices and scaling indices of elements from row level to global level in the tensor
            base_indices=torch.arange(original_batch_size,device=device).unsqueeze(1) * num_beams
            scaled_beam_indices=beam_indices+base_indices #Tensor of shape shape (original_batch_size, num_beam)
            flattened_scaled_beam_indices=scaled_beam_indices.view(-1) #making a 1D tensor of shape(original_batch_size*num_beam)

            #By Advance Indexing using these scaled indices we will obtain sequences from decoder_input_ids of the respective beam that the indices point to
            prev_decoder_inputs=decoder_input_ids[flattened_scaled_beam_indices]

            #Getting the token_ids of the tokens to be added
            reshaped_top_total_token_ids=total_top_token_ids.view(original_batch_size, num_beams*num_beams)
            """
            Here each row will contain the token ids of the top scoring tokens of all beams of a sequence just like the structure of reshaped_total_top_score
            Since new_token_indices tells the row index of the topk elements of that row, the same indices will also point to the token_ids in reshaped_top_total_token_ids
            So we make a vector of shape (original_batch_size,1) where each element will help us point to a row indices in reshaped_top_total_token_ids
            And then new_token_indices has column indices which help us point to columns of reshaped_top_total_token_ids for that row
            in this case of Advacne indexing because we use two rows and columns we get a new tensor of shape of rows and columns used
            """

            raw_new_tokens = reshaped_top_total_token_ids[torch.arange(original_batch_size,device=device).unsqueeze(1), new_token_indices] #the token_ids for the new tokens in a Tensor of shape(original_batch_size, num_beams)
            new_tokens = raw_new_tokens.view(original_batch_size*num_beams,1) #2D Tensor of shape(original_batch_size*num_beam,1) made to be suited for concatenation to decoder_input_ids
            decoder_input_ids=torch.cat([prev_decoder_inputs, new_tokens], dim=1)

            #Updating the beam scores to be used after this step in generation
            beam_scores=new_top_scores.view(-1)

            #Printing decoder_input_ids to see whats going on
            #print(f"Decoder input ids at step{step}:\n{decoder_input_ids}")

            # Check for early stopping
            if early_stopping:
                # If all beams for all sequences are finished, stop decoding
                if new_tokens[0]==tokenizer.eos_token_id: #if the top beam at the time generates the EOS token then break the loop
                    break


    # Reshape and return the best sequences
    output_sequences = decoder_input_ids.view(original_batch_size, num_beams, -1) # creates a 3D tensor with original_batch_size number of block, and each block containing num_beam rows, and each rows has lenght=sequence_length

    # Select the top beam for each batch
    best_sequences = output_sequences[:, 0, :] #slices to create a 2D tensor of shape(original_batch_size, sequence_lenght) which selects first row of each block since the first row would be the one with highest total score.

    return best_sequences


def translate(dataset):
    #tokenize the dataset at once because it is a small dataset and tokenization does not need to be done in batches
    dataset_t = tokenizer(
                [row["en"] for row in dataset["translation"]],
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128
                ).to(device) #we move the tensors to the device that our model is on

    # Generate translation for batches of tokenized dataset and add them to translated_dataset list, we do this in batches to speed up the process by making use of the parallel procesing but we dont do it all at once because of possible memory constraints.
    input_ids=dataset_t["input_ids"] #This will be a 2D tensor of shape(total number of sequences tokenized, i.e. length of list passed in tokenizer, sequence length)
    attention_mask=dataset_t["attention_mask"] #This will be 2D tensor of shape(total number of sequences tokenized, i.e. length of list passed in tokenizer, sequence length)
    translated_dataset=[] #The list of generated translation which will be returned from the function
    for i in tqdm(range(0,input_ids.shape[0], generation_batch_size)):
        batch_input_ids= input_ids[i:i+generation_batch_size] #slicing for tensors works in the same way as slicing for lists, so if i+generation_batch_size>input_ids.shape[0] then slicing will just return a tensor until the last element of the parent tensor.
        batch_attention_mask=attention_mask[i:i+generation_batch_size]
        outputs = scratch_beam_search(input_ids=batch_input_ids, attention_mask=batch_attention_mask, num_beams=4, early_stopping=True)
        translated_batch=tokenizer.batch_decode(outputs, skip_special_tokens=True) # tokenizer.decode could also handle 2D tensor containing muliple token_ids but .batch_decode is said to handle it faster
        translated_dataset.extend(translated_batch) # .extend adds each element from an iterable to a list, whereas if we used .append here it would have added the entire iterable to the list as a single element

    return translated_dataset

# Translate all EN sentences
model_generated_list = translate(eval_dataset)


#BLEU score for model generation
bleu_score = sacrebleu.corpus_bleu(model_generated_list, reference_list_BLEU)
print(f"BLEU score for validation split using scratch_beam_search:{bleu_score}")



100%|██████████| 2818/2818 [21:27<00:00,  2.19it/s]


BLEU score for validation split using scratch_beam_search:BLEU = 27.48 57.4/33.3/21.3/14.0 (BP = 1.000 ratio = 1.012 hyp_len = 56568 ref_len = 55920)


## Section 4: Implementing Lexical Constraints for 4 Iterations:

##### Plan for Implementing Lexical constraints:
1. The baseline is ready for implementing constraints

2. As you can see I can get the model_generated_list which contains the list of translated sequences for entire dataset by the translate function(which further calls the scratch_beam_search function).

3. Now I define a contrained_translate function that tokenizes the model_generated_list and target_reference list.Then runs a loop in which it calls find_missing_tokens function to return the missing tokens for the sequence in current iteration. Then it calls constrained_beam_search to include the constraint in the translation for that sequence. Then it adds the translation of that sequence to the constrained_translated_list.

4. Then We calculte the BLEU score for the all 4 iterations of constrained_translation 

In [ ]:
## Implementing Lexical Constraints Once(First iteration)
eval_dataset=dataset["validation"]
source_reference_list=[row["en"] for row in eval_dataset["translation"]] #A list of all english sentences in the dataset
target_reference_list=[row["ru"] for row in eval_dataset["translation"]] #A list of all russian sentence in the dataset
reference_list_BLEU=[[row["ru"] for row in eval_dataset["translation"]]]
generation_batch_size=1 #increase or decrease as per memory available in models processing unit
model.eval()# Putting the model in eval mode
GBS_constraints=[[] for _ in range(len(source_reference_list))] # we make a list containing len(source_reference_list) empty lists where each empty list will contain contraints for each sequence in the future This will be list of lists that contain subsequences(lexical constraints) for each input sequence in eval_dataset,

"""
find_missing_tokens function returns the n-gram(subsequence) from reference_seq which is missing from translated_sequence,
It looks for 3-grams, if none found then for 2-grams, if none found then 1-grams, and return None if not even 1-grams are found to be missing from translated_sequence
"""
def find_missing_tokens(translated_seq, reference_seq):
    #This nested function returns a boolean value after checking if the subsequence is missing from the translated_seq
    def is_subsequence_missing(subsequence, translated_seq, n):
        #Goes over all n-grams(subsequences) in the translated_seq
        for i in range(len(translated_seq) - n + 1):
            #compares all n-grams in translated_seq and
            if torch.equal(translated_seq[i:i + n], subsequence): #if one of the subsequences in translated_seq is equal to subsequence it returns false because it means subsequence is not missing from translated_seq
                return False
        return True

    # Check for missing subsequences of in the order 3, 2, and 1 tokens
    for n in range(3, 0, -1):
        #Goes over all n-grams(subsequences) in the reference_seq
        for i in range(len(reference_seq) - n + 1): #number of n-grams in a sequence=len_of_seq-n+1, here len(reference_seq) give the length of the reference sequence since it is a 1D tensor
            subsequence = reference_seq[i:i + n] #choosing the current n-gram
            if is_subsequence_missing(subsequence, translated_seq, n): #returns True if current n-gram is missing
                return subsequence, i #basically returns the first missing n-gram we can find in the order of looking for 3,2,1-grams and the position of the first token of the n-gram in reference

    # If no missing subsequences are found, return None
    return None, None


def constrained_beam_search(input_ids, attention_mask, constraints, num_beams=4, max_length=128, early_stopping=True):
    original_batch_size = input_ids.shape[0] #will be 1, because input_ids here is a 1D tensor in case we are only sending in 1 sequence from constrained_translate() to be translated in constrained_beam_search_ at a time.

    #scaling original input to num_beams for beam search
    input_ids = input_ids.repeat_interleave(num_beams, dim=0) #creating num_beam copies the each input sequence(one for each beam).
    attention_mask = attention_mask.repeat_interleave(num_beams, dim=0) #doing the same thing again but for attention masks

    # Creating tensor for storing outputs of current beams
    decoder_input_ids = torch.full(  #making a 2D tensor of shape (original_batch_size * num_beams, 1) with each element equal to "BOS" token_id of the decoder/tokenizer
                        (original_batch_size * num_beams, 1),
                        model.config.decoder_start_token_id,
                        dtype=torch.long,
                        device=device
                        )

    # making a 1D tensor for storing scores of all beams of all sequences
    beam_scores = torch.zeros(original_batch_size * num_beams, dtype=torch.float, device=device)

    #Maintains a list of states for each beam
    constraint_states = [[{"active": True, "satisfied": False, "tokens_left": constraint_info["token_ids"].clone(), "position": constraint_info["position"]} for constraint_info in constraints] for _ in range(original_batch_size * num_beams)] #list for each beam of all input sequences, each beam(row) has a list of constraints states
                                                                                                                             #each constraint state is a dictionary with 3 keys- "active"(meaning it is actively looking to be fullfilled), "satisfied"(meaning it is satisfied), "tokens_left"(which contain a copy of the tokens to be included for that constraint in that beam)
    Start=True ##initial Condition to be checked only once during the first forward pass                                                                                                                           #Active=True means the full constraint is pending or part of the constraint is pending
    #Helper function for adding constraint token
    def find_variable_in_constraints(step, constraints):
        position_list=[]
        for i, constraint_dict in enumerate(constraints):
            if step == constraint_dict["position"] and constraint_dict["active"]==True: #Constraint must be active constraint
                position_list.append(i)  # Return the index of the dictionary whose constraint position is equal to current decoding step
        if position_list: #if their is anything in the position list
            return position_list
        return None     #Else return None
    #Decoding Steps for generating the output
    for step in range(max_length):#each decoding step, adding one token to a beam of decoder_input_ids

        #Adding Constraint
        constraint_num=find_variable_in_constraints(step, constraint_states[0]) #checking for position in first beam since all beams are updated together anyways, it will return constraints whose "position" is equal to the current step
        if constraint_num is not None: #this step is equal to position of some constraint
            #add constraint token to decoder_input_id
            constraint_token_tensor=torch.full((original_batch_size * num_beams, 1), constraint_states[0][constraint_num[0]]["tokens_left"][0],dtype=torch.long, device=device)
            """
            Explaination of above step:
            constraint_states[0] gives us a list of dictionaries containing constraint info for all constraints of the first beam(since all constraints for all beams are updated together we can take any one, it doesnt matter)
            then from the constraint_num list that has a list of indices for all constraints that have first token valid for this step, we choose the first one i.e.constraint_num[0](it wouldnt matter if you choose the second one too since the token for a specific position would be the same as they are all derived from the same reference translation)
            Then we take the first token of that constraint and prepare a tensor to add it to the decoder input_ids
            """
            #add the 1 token of constraint to beams of decoder_input_id
            decoder_input_ids=torch.cat([decoder_input_ids, constraint_token_tensor], dim=1)


            #update constraint_state for beams of the sequence
            for i in range(original_batch_size*num_beams): #for all beams
                for x in range(len(constraint_num)):  #for all constraints with position same as this step
                    constraint_states[i][constraint_num[x]]["position"]+=1 #shift position 1 index forward for the next token
                    constraint_states[i][constraint_num[x]]["tokens_left"]=constraint_states[i][constraint_num[x]]["tokens_left"][1:] #remove the token from the constraint that has been added to the decoder_input_id
                    if len(constraint_states[i][constraint_num[x]]["tokens_left"])==0: #when there are no more tokens left in that constraint
                        constraint_states[i][constraint_num[x]]["active"]=False #change the state of the constraint



        #Generating token by use of logits and Scratch beam search
        else: #this step is not a position of some constraint then we just generate using models logits
            with torch.no_grad():

                # Forward pass
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    decoder_input_ids=decoder_input_ids
                    ) #This will be a dictionary like object containing a lot of things like hidden states,etc, but we are most interested in logits
                """
                Talking about some important Tensor Operations:

                Broadcasting:
                In pytorch, a process called broadcasting happens just before an element wise operation that requires two tensors with same number of dimensions to align in shape.
                Example Adding tensor A of shape(4,6,8) with tensor B of shape(4,1,1) then the scaler values in tensor B along the dimensions with only 1 element are duplicated to match the shape of the tensor A.
                But broadcasting only happens when two tensors have same size of some dimensions and some dimensions of size 1.

                tensor.view(size of dim=0, size of dim=1.....) is used to create a reshaped tensor with the same number of elements as the original tensor.
                It can add new dimensions of a specific size as long as the number of element in the original tensor could be accomodated.
                tensor.view(-1,10) tell us to accomodate 10 elements in dim=10 and infer what number of element to accomodate in dim=0 such that the total number of elements remain the same.

                tensor.unsqueeze() is used to add a singleton dimension(dimension of size 1) at a specified index
                tensor.unsequeeze(1) adds a singleton dimension at index=1

                tensor.squeeze() is used remove a singleton dimension from a specified index

                tensor.expand() is used to basically perform broadcasting on command, you would send in sizes of dimension of tensor which are capable of broadcasting and get a result.
                This means that the size you send in should either be the same as sizes of dimension of original tensor OR in case of singleton dimension in original tensor you could send in a greater number to stretch along that dimension
                tensor.expand(-1, 4) tells to keep dim=0 as is and basically stretch dim=1 to size 4, this is conditioned on the fact that the dimension to be stretched is of size 1 otherwise an error will occur

                tensor.expand_as(target_tensor) is used to perform broadcasting on tensor conditioned on the fact that the size of target_tensor is suitable for broadcasting


                """
                # Creating a 2D tensor Logits
                logits = outputs.logits[:, -1, :] #output.logits is a 3-D tensor of shape(original_batch_size*num_beam, current sequence length in decoder_input_ids, vocab size). where each element is the logit of each token_id of each sequence for each element of the vocabulary
                                                #By slicing we choose the last token_id of each sequence it to get a 2-D tensor of shape(original_batch_size*num_beam, vocab_size) where each element is a logit of last token_id of each sequence element for each element of the vocabulary
                #print(f"Logits at step {step}:\n{logits}")
                # Using Boolean masking to mark finished beams (those that already generated EOS)
                #finished_mask = finished_beams.view(-1, 1).expand(-1, logits.size(-1))  # finished_mask will be a 2D of shape(original_batch_size * num_beams, vocab_size) with values of True/False repeated in each row(representing each beam) upto the vocab_size
                #logits[finished_mask] = -float('inf')  # Making logits of lasts token for all elements of vocabulary for the finished beams negative infinity, By sending in boolean mask tensor of the same shape as the logits tensor the operation is applied to all logits elements that correspond with True in finished mask.
                                                        #Making Logit for all words in vocabulary negative infinity means probability=0 after softmax, and log of that is a big negative value, when that big negative value is combined with beam_score to get next_score it ensures that these beams will now not come in the topk.
                                                        #This ensures that only active beams (those not yet finished) are considered during the top-k selection.

                # Creating 2D tensor log probabilities
                log_probs = F.log_softmax(logits, dim=-1) #performs softmax on all logits normalizing them to probability of 0-1, then takes log of that so the highest probability is turend into the largest number(smallest negative value).
                                                        #dim=-1 takes softmax along the last dimension(2nd dimension with size=vocab_size in this case) which means that normalization happens for each row(last token for each sequence of decoder_input_ids)
                #print(f"Log_probs at step {step}:\n{log_probs}")
                #get top num_beams scores from each beam
                total_top_log_probs, total_top_token_ids=torch.topk(log_probs, num_beams, dim=1) #give 2 tensors of shape(original_batch_size*num_beams, num_beams) where each row contains scores top num_beam scores for that row and the corresponding token_ids in descending
                #print(f"Topk log_probs at step {step}:\n{total_top_log_probs}")
                #print(f"Topk token_ids at step {step}:\n{total_top_token_ids}")

                if Start==True:
                    new_tokens=total_top_token_ids[::num_beams].reshape(-1,1) #using slicing with step value num_beam we take the top 4 tokens of a beam of a sequence and make a tensor containing new tokens for each beam, .reshape is used in place of .view() becauce we used slicing and so tensor is not contiguous in memory hence .view wont work
                    decoder_input_ids=torch.cat([decoder_input_ids[torch.arange(original_batch_size*num_beams)],new_tokens], dim=1) #adding first 4 tokens to different beam of each sequence
                    beam_scores=total_top_log_probs[::num_beams].reshape(-1) #slicing to make tensor of shape(original_batch_size,num_beams) and then we flatten it using .reshape(-1)
                    Start=False
                    #print(f"Decoder_input_ids at step{step}:\n{decoder_input_ids}")
                    #print(f"Beam score at step{step}:\n{beam_scores}")

                    continue

                #calculate total scores for each beams top 4 tokens
                total_top_score=total_top_log_probs+beam_scores.unsqueeze(1)
                #print(f"Total score of each topk token of each beam scores at step {step}:\n{total_top_score}")


                #reshape that
                reshaped_total_top_score=total_top_score.view(original_batch_size, num_beams*num_beams)

                #Getting the best scores for this step in generation
                new_top_scores,new_token_indices=torch.topk(reshaped_total_top_score,num_beams, dim=1) #gives 2 tensors of shape (original_batch_size, num_beam) containing the top num_beam score from all total_scores of one sequence in each row in descending order and their index in that row in total_top_score_reshaped
                #print(f"Total Top scores for new tokens of each sequence scores at step {step}:\n{new_top_scores}")
                #calculating beam indices for the new tokens of each sequence with best score
                beam_indices=new_token_indices//num_beams #elements of a row in this tensor will tell which token originally belonged to which beam for a sequence
                #reshaping beam_indices and scaling indices of elements from row level to global level in the tensor
                base_indices=torch.arange(original_batch_size,device=device).unsqueeze(1) * num_beams
                scaled_beam_indices=beam_indices+base_indices #Tensor of shape shape (original_batch_size, num_beam)
                flattened_scaled_beam_indices=scaled_beam_indices.view(-1) #making a 1D tensor of shape(original_batch_size*num_beam)

                #By Advance Indexing using these scaled indices we will obtain sequences from decoder_input_ids of the respective beam that the indices point to
                prev_decoder_inputs=decoder_input_ids[flattened_scaled_beam_indices]

                #Getting the token_ids of the tokens to be added
                reshaped_top_total_token_ids=total_top_token_ids.view(original_batch_size, num_beams*num_beams)
                """
                Here each row will contain the token ids of the top scoring tokens of all beams of a sequence just like the structure of reshaped_total_top_score
                Since new_token_indices tells the row index of the topk elements of that row, the same indices will also point to the token_ids in reshaped_top_total_token_ids
                So we make a vector of shape (original_batch_size,1) where each element will help us point to a row indices in reshaped_top_total_token_ids
                And then new_token_indices has column indices which help us point to columns of reshaped_top_total_token_ids for that row
                in this case of Advacne indexing because we use two rows and columns we get a new tensor of shape of rows and columns used
                """

                raw_new_tokens = reshaped_top_total_token_ids[torch.arange(original_batch_size,device=device).unsqueeze(1), new_token_indices] #the token_ids for the new tokens in a Tensor of shape(original_batch_size, num_beams)
                new_tokens = raw_new_tokens.view(original_batch_size*num_beams,1) #2D Tensor of shape(original_batch_size*num_beam,1) made to be suited for concatenation to decoder_input_ids
                decoder_input_ids=torch.cat([prev_decoder_inputs, new_tokens], dim=1)

                #Updating the beam scores to be used after this step in generation
                beam_scores=new_top_scores.view(-1)

                #Printing decoder_input_ids to see whats going on
                #print(f"Decoder input ids at step{step}:\n{decoder_input_ids}")

                # eos_mask = (new_tokens == tokenizer.eos_token_id) #creating 2D boolean of shape as new_tokens telling which tokens generated now are EOS by setting that value to True
                # finished_beams = finished_beams | eos_mask.view(-1) #.view(-1)Flattens eos_mask to make it a 1D tensor of shape(original_batch_size*num_beams) same as finished_beams, then we perform an OR operation(|) to update finished beam to True for the beams that have just generated EOS tokens

                # Check for early stopping
                if early_stopping:
                    # If all beams for all sequences are finished, stop decoding
                    if new_tokens[0]==tokenizer.eos_token_id: #if the top beam at the time generates the EOS token then break the loop
                        break


    # Reshape and extract the best sequence for each input
    output_sequences = decoder_input_ids.view(original_batch_size, num_beams, -1)
    best_sequences = output_sequences[:, 0, :] #creates 2D tensor of shape(original_batch_size, sequence length)

    return best_sequences


# Requires a translated list(list containing MT hypothesis) of previous iteration and index of current iteration
def constrained_translate(translated_list):
    #Tokenizing translated list

    translated_list_t = tokenizer(
                translated_list,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128
                ).to(device)
    translated_input_ids=translated_list_t["input_ids"] #This will be a 2D tensor of shape(total number of sequences tokenized, sequence length)
    translated_attention_mask=translated_list_t["attention_mask"] #This will be 2D tensor of shape(total number of sequences tokenized, sequence length)

    dataset_t = tokenizer(
                source_reference_list,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128
                ).to(device)
    source_input_ids=dataset_t["input_ids"] #This will be a 2D tensor of shape(total number of sequences tokenized, i.e. length of list passed in tokenizer, sequence length)
    source_attention_mask=dataset_t["attention_mask"] #This will be 2D tensor of shape(total number of sequences tokenized, i.e. length of list passed in tokenizer, sequence length)


    #Tokenizing target(russian) reference list
    reference_list_t = tokenizer(
            target_reference_list,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
            ).to(device)

    reference_input_ids = reference_list_t["input_ids"]  # This will be 2D tensor of shape(total number of sequences tokenized, sequence length)
    reference_attention_mask = reference_list_t["attention_mask"]  # This will be 2D tensor of shape(total number of sequences tokenized, sequence length)

    constrained_translated_list=[]


    # Iterating through each translated sequence
    for i in tqdm(range(len(translated_list))):
        translated_seq_ids = translated_input_ids[i:i+1]  # 2D tensor of shape [1, max seq_length in the tokenized list] containing token_ids for Single sequence from translated batch
        reference_seq_ids = reference_input_ids[i:i+1]  # 2D tensor of shape [1, max seq_length in the tokenized list] containing token_ids for Corresponding reference sequence
        source_seq_ids= source_input_ids[i:i+1] # 2D tensor of shape [1, max seq_length in the tokenized list] containing token_ids for Corresponding reference sequence
        source_seq_attention_mask=source_attention_mask[i:i+1]


        # Find phrases(subsequences) missing from translated_seq to add them as constraints
        constraint, position = find_missing_tokens(translated_seq_ids[0], reference_seq_ids[0])#constraint will be a 1D array of token_ids, Sending in values of the one row in translated_seq_ids and reference_seq_ids in find_missing_tokens, so these are 1D tensors(imagine like a list of integers)
                                                                                    #it could also return None in case of no mismatch

        #In case there is no mismatch in translated and reference list just add the sequence from translated_list in the constrained_translated_list, and continue to next sequence
        if constraint==None:
            constrained_translated_list.append(translated_list[i]) #.append will add the string as a single element to constrained_translated_list, if we use .extend here it will treat the string as an iterable and add each of its elements(characters) to the constrained_translated_list
            continue

        constraint_info={"token_ids":constraint, "position":position}
        #Add the constraint to the list of constraints for the current sequence
        GBS_constraints[i].append(constraint_info)


        #Generated translation using Grid_beam_search
        constrained_seq_ids = constrained_beam_search(
            input_ids=source_seq_ids,#sending input_ids of one sequence
            attention_mask=source_seq_attention_mask, #sending attention mask of 1 sequence
            constraints=GBS_constraints[i], #Sending constraints for that one sequence
        )


        constrained_translation = tokenizer.batch_decode(constrained_seq_ids, skip_special_tokens=True)

        constrained_translated_list.extend(constrained_translation) #we use .extend here because it will add each element of the iterable constrained_translation to constrained_translated_list, constrained_translation could be a list of strings, but in this case is just a list containing one string

    return constrained_translated_list


#Getting the translation of input sequence by deriving constraints for each sequence of target_reference_list subsequences that are missing from model_generated_list
constrained_translation_1=constrained_translate(model_generated_list)  #list of Constrained Translated Sequnces in Target Language


#BLEU score for fist iteration
Base_blue_score= sacrebleu.corpus_bleu(model_generated_list, reference_list_BLEU)
print(f"\nBase Bleu score:{Base_blue_score} ")
#Confirm number of elements in hypotheses list is equal to number of sequences in reference list)
if len(constrained_translation_1)==len(reference_list_BLEU[0]):
    constrained_bleu_score1= sacrebleu.corpus_bleu(constrained_translation_1, reference_list_BLEU)
    print(f"\n Constrained Bleu Score for 1st iteration:{constrained_bleu_score1}")

else:
    print("Mismatch in number of hypotheses and reference sequences")



100%|██████████| 2818/2818 [22:31<00:00,  2.08it/s]



Base Bleu score:BLEU = 27.48 57.4/33.3/21.3/14.0 (BP = 1.000 ratio = 1.012 hyp_len = 56568 ref_len = 55920) 

 Constrained Bleu Score:BLEU = 26.34 56.7/32.2/20.2/13.0 (BP = 1.000 ratio = 1.005 hyp_len = 56172 ref_len = 55920)


In [ ]:
## Implementing Lexical Constraints (Second iteration)
eval_dataset=dataset["validation"]
source_reference_list=[row["en"] for row in eval_dataset["translation"]] #A list of all english sentences in the dataset
target_reference_list=[row["ru"] for row in eval_dataset["translation"]] #A list of all russian sentence in the dataset
reference_list_BLEU=[[row["ru"] for row in eval_dataset["translation"]]]
generation_batch_size=1 #increase or decrease as per memory available in models processing unit
model.eval()# Putting the model in eval mode
GBS_constraints_2=GBS_constraints

"""
find_missing_tokens function returns the n-gram(subsequence) from reference_seq which is missing from translated_sequence,
It looks for 3-grams, if none found then for 2-grams, if none found then 1-grams, and return None if not even 1-grams are found to be missing from translated_sequence
"""
def find_missing_tokens(translated_seq, reference_seq):
    #This nested function returns a boolean value after checking if the subsequence is missing from the translated_seq
    def is_subsequence_missing(subsequence, translated_seq, n):
        #Goes over all n-grams(subsequences) in the translated_seq
        for i in range(len(translated_seq) - n + 1):
            #compares all n-grams in translated_seq and
            if torch.equal(translated_seq[i:i + n], subsequence): #if one of the subsequences in translated_seq is equal to subsequence it returns false because it means subsequence is not missing from translated_seq
                return False
        return True

    # Check for missing subsequences of in the order 3, 2, and 1 tokens
    for n in range(3, 0, -1):
        #Goes over all n-grams(subsequences) in the reference_seq
        for i in range(len(reference_seq) - n + 1): #number of n-grams in a sequence=len_of_seq-n+1, here len(reference_seq) give the length of the reference sequence since it is a 1D tensor
            subsequence = reference_seq[i:i + n] #choosing the current n-gram
            if is_subsequence_missing(subsequence, translated_seq, n): #returns True if current n-gram is missing
                return subsequence, i #basically returns the first missing n-gram we can find in the order of looking for 3,2,1-grams and the position of the first token of the n-gram in reference

    # If no missing subsequences are found, return None
    return None, None

def constrained_beam_search(input_ids, attention_mask, constraints, num_beams=4, max_length=128, early_stopping=True):
    original_batch_size = input_ids.shape[0] #will be 1, because input_ids here is a 1D tensor in case we are only sending in 1 sequence from constrained_translate() to be translated in constrained_beam_search_ at a time.

    #scaling original input to num_beams for beam search
    input_ids = input_ids.repeat_interleave(num_beams, dim=0) #creating num_beam copies the each input sequence(one for each beam).
    attention_mask = attention_mask.repeat_interleave(num_beams, dim=0) #doing the same thing again but for attention masks

    # Creating tensor for storing outputs of current beams
    decoder_input_ids = torch.full(  #making a 2D tensor of shape (original_batch_size * num_beams, 1) with each element equal to "BOS" token_id of the decoder/tokenizer
                        (original_batch_size * num_beams, 1),
                        model.config.decoder_start_token_id,
                        dtype=torch.long,
                        device=device
                        )

    # making a 1D tensor for storing scores of all beams of all sequences
    beam_scores = torch.zeros(original_batch_size * num_beams, dtype=torch.float, device=device)

    #Maintains a list of states for each beam
    constraint_states = [[{"active": True, "satisfied": False, "tokens_left": constraint_info["token_ids"].clone(), "position": constraint_info["position"]} for constraint_info in constraints] for _ in range(original_batch_size * num_beams)] #list for each beam of all input sequences, each beam(row) has a list of constraints states
                                                                                                                             #each constraint state is a dictionary with 3 keys- "active"(meaning it is actively looking to be fullfilled), "satisfied"(meaning it is satisfied), "tokens_left"(which contain a copy of the tokens to be included for that constraint in that beam)
    Start=True ##initial Condition to be checked only once during the first forward pass                                                                                                                           #Active=True means the full constraint is pending or part of the constraint is pending
    #Helper function for adding constraint token
    def find_variable_in_constraints(step, constraints):
        position_list=[]
        for i, constraint_dict in enumerate(constraints):
            if step == constraint_dict["position"] and constraint_dict["active"]==True: #Constraint must be active constraint
                position_list.append(i)  # Return the index of the dictionary whose constraint position is equal to current decoding step
        if position_list: #if their is anything in the position list
            return position_list
        return None     #Else return None
    #Decoding Steps for generating the output
    for step in range(max_length):#each decoding step, adding one token to a beam of decoder_input_ids

        #Adding Constraint
        constraint_num=find_variable_in_constraints(step, constraint_states[0]) #checking for position in first beam since all beams are updated together anyways, it will return constraints whose "position" is equal to the current step
        if constraint_num is not None: #this step is equal to position of some constraint
            #add constraint token to decoder_input_id
            constraint_token_tensor=torch.full((original_batch_size * num_beams, 1), constraint_states[0][constraint_num[0]]["tokens_left"][0],dtype=torch.long, device=device)
            """
            Explaination of above step:
            constraint_states[0] gives us a list of dictionaries containing constraint info for all constraints of the first beam(since all constraints for all beams are updated together we can take any one, it doesnt matter)
            then from the constraint_num list that has a list of indices for all constraints that have first token valid for this step, we choose the first one i.e.constraint_num[0](it wouldnt matter if you choose the second one too since the token for a specific position would be the same as they are all derived from the same reference translation)
            Then we take the first token of that constraint and prepare a tensor to add it to the decoder input_ids
            """
            #add the 1 token of constraint to beams of decoder_input_id
            decoder_input_ids=torch.cat([decoder_input_ids, constraint_token_tensor], dim=1)


            #update constraint_state for beams of the sequence
            for i in range(original_batch_size*num_beams): #for all beams
                for x in range(len(constraint_num)):  #for all constraints with position same as this step
                    constraint_states[i][constraint_num[x]]["position"]+=1 #shift position 1 index forward for the next token
                    constraint_states[i][constraint_num[x]]["tokens_left"]=constraint_states[i][constraint_num[x]]["tokens_left"][1:] #remove the token from the constraint that has been added to the decoder_input_id
                    if len(constraint_states[i][constraint_num[x]]["tokens_left"])==0: #when there are no more tokens left in that constraint
                        constraint_states[i][constraint_num[x]]["active"]=False #change the state of the constraint



        #Generating token by use of logits and Scratch beam search
        else: #this step is not a position of some constraint then we just generate using models logits
            with torch.no_grad():

                # Forward pass
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    decoder_input_ids=decoder_input_ids
                    ) #This will be a dictionary like object containing a lot of things like hidden states,etc, but we are most interested in logits
                """
                Talking about some important Tensor Operations:

                Broadcasting:
                In pytorch, a process called broadcasting happens just before an element wise operation that requires two tensors with same number of dimensions to align in shape.
                Example Adding tensor A of shape(4,6,8) with tensor B of shape(4,1,1) then the scaler values in tensor B along the dimensions with only 1 element are duplicated to match the shape of the tensor A.
                But broadcasting only happens when two tensors have same size of some dimensions and some dimensions of size 1.

                tensor.view(size of dim=0, size of dim=1.....) is used to create a reshaped tensor with the same number of elements as the original tensor.
                It can add new dimensions of a specific size as long as the number of element in the original tensor could be accomodated.
                tensor.view(-1,10) tell us to accomodate 10 elements in dim=10 and infer what number of element to accomodate in dim=0 such that the total number of elements remain the same.

                tensor.unsqueeze() is used to add a singleton dimension(dimension of size 1) at a specified index
                tensor.unsequeeze(1) adds a singleton dimension at index=1

                tensor.squeeze() is used remove a singleton dimension from a specified index

                tensor.expand() is used to basically perform broadcasting on command, you would send in sizes of dimension of tensor which are capable of broadcasting and get a result.
                This means that the size you send in should either be the same as sizes of dimension of original tensor OR in case of singleton dimension in original tensor you could send in a greater number to stretch along that dimension
                tensor.expand(-1, 4) tells to keep dim=0 as is and basically stretch dim=1 to size 4, this is conditioned on the fact that the dimension to be stretched is of size 1 otherwise an error will occur

                tensor.expand_as(target_tensor) is used to perform broadcasting on tensor conditioned on the fact that the size of target_tensor is suitable for broadcasting


                """
                # Creating a 2D tensor Logits
                logits = outputs.logits[:, -1, :] #output.logits is a 3-D tensor of shape(original_batch_size*num_beam, current sequence length in decoder_input_ids, vocab size). where each element is the logit of each token_id of each sequence for each element of the vocabulary
                                                #By slicing we choose the last token_id of each sequence it to get a 2-D tensor of shape(original_batch_size*num_beam, vocab_size) where each element is a logit of last token_id of each sequence element for each element of the vocabulary
                #print(f"Logits at step {step}:\n{logits}")
                # Using Boolean masking to mark finished beams (those that already generated EOS)
                #finished_mask = finished_beams.view(-1, 1).expand(-1, logits.size(-1))  # finished_mask will be a 2D of shape(original_batch_size * num_beams, vocab_size) with values of True/False repeated in each row(representing each beam) upto the vocab_size
                #logits[finished_mask] = -float('inf')  # Making logits of lasts token for all elements of vocabulary for the finished beams negative infinity, By sending in boolean mask tensor of the same shape as the logits tensor the operation is applied to all logits elements that correspond with True in finished mask.
                                                        #Making Logit for all words in vocabulary negative infinity means probability=0 after softmax, and log of that is a big negative value, when that big negative value is combined with beam_score to get next_score it ensures that these beams will now not come in the topk.
                                                        #This ensures that only active beams (those not yet finished) are considered during the top-k selection.

                # Creating 2D tensor log probabilities
                log_probs = F.log_softmax(logits, dim=-1) #performs softmax on all logits normalizing them to probability of 0-1, then takes log of that so the highest probability is turend into the largest number(smallest negative value).
                                                        #dim=-1 takes softmax along the last dimension(2nd dimension with size=vocab_size in this case) which means that normalization happens for each row(last token for each sequence of decoder_input_ids)
                #print(f"Log_probs at step {step}:\n{log_probs}")
                #get top num_beams scores from each beam
                total_top_log_probs, total_top_token_ids=torch.topk(log_probs, num_beams, dim=1) #give 2 tensors of shape(original_batch_size*num_beams, num_beams) where each row contains scores top num_beam scores for that row and the corresponding token_ids in descending
                #print(f"Topk log_probs at step {step}:\n{total_top_log_probs}")
                #print(f"Topk token_ids at step {step}:\n{total_top_token_ids}")

                if Start==True:
                    new_tokens=total_top_token_ids[::num_beams].reshape(-1,1) #using slicing with step value num_beam we take the top 4 tokens of a beam of a sequence and make a tensor containing new tokens for each beam, .reshape is used in place of .view() becauce we used slicing and so tensor is not contiguous in memory hence .view wont work
                    decoder_input_ids=torch.cat([decoder_input_ids[torch.arange(original_batch_size*num_beams)],new_tokens], dim=1) #adding first 4 tokens to different beam of each sequence
                    beam_scores=total_top_log_probs[::num_beams].reshape(-1) #slicing to make tensor of shape(original_batch_size,num_beams) and then we flatten it using .reshape(-1)
                    Start=False
                    #print(f"Decoder_input_ids at step{step}:\n{decoder_input_ids}")
                    #print(f"Beam score at step{step}:\n{beam_scores}")

                    continue

                #calculate total scores for each beams top 4 tokens
                total_top_score=total_top_log_probs+beam_scores.unsqueeze(1)
                #print(f"Total score of each topk token of each beam scores at step {step}:\n{total_top_score}")


                #reshape that
                reshaped_total_top_score=total_top_score.view(original_batch_size, num_beams*num_beams)

                #Getting the best scores for this step in generation
                new_top_scores,new_token_indices=torch.topk(reshaped_total_top_score,num_beams, dim=1) #gives 2 tensors of shape (original_batch_size, num_beam) containing the top num_beam score from all total_scores of one sequence in each row in descending order and their index in that row in total_top_score_reshaped
                #print(f"Total Top scores for new tokens of each sequence scores at step {step}:\n{new_top_scores}")
                #calculating beam indices for the new tokens of each sequence with best score
                beam_indices=new_token_indices//num_beams #elements of a row in this tensor will tell which token originally belonged to which beam for a sequence
                #reshaping beam_indices and scaling indices of elements from row level to global level in the tensor
                base_indices=torch.arange(original_batch_size,device=device).unsqueeze(1) * num_beams
                scaled_beam_indices=beam_indices+base_indices #Tensor of shape shape (original_batch_size, num_beam)
                flattened_scaled_beam_indices=scaled_beam_indices.view(-1) #making a 1D tensor of shape(original_batch_size*num_beam)

                #By Advance Indexing using these scaled indices we will obtain sequences from decoder_input_ids of the respective beam that the indices point to
                prev_decoder_inputs=decoder_input_ids[flattened_scaled_beam_indices]

                #Getting the token_ids of the tokens to be added
                reshaped_top_total_token_ids=total_top_token_ids.view(original_batch_size, num_beams*num_beams)
                """
                Here each row will contain the token ids of the top scoring tokens of all beams of a sequence just like the structure of reshaped_total_top_score
                Since new_token_indices tells the row index of the topk elements of that row, the same indices will also point to the token_ids in reshaped_top_total_token_ids
                So we make a vector of shape (original_batch_size,1) where each element will help us point to a row indices in reshaped_top_total_token_ids
                And then new_token_indices has column indices which help us point to columns of reshaped_top_total_token_ids for that row
                in this case of Advacne indexing because we use two rows and columns we get a new tensor of shape of rows and columns used
                """

                raw_new_tokens = reshaped_top_total_token_ids[torch.arange(original_batch_size,device=device).unsqueeze(1), new_token_indices] #the token_ids for the new tokens in a Tensor of shape(original_batch_size, num_beams)
                new_tokens = raw_new_tokens.view(original_batch_size*num_beams,1) #2D Tensor of shape(original_batch_size*num_beam,1) made to be suited for concatenation to decoder_input_ids
                decoder_input_ids=torch.cat([prev_decoder_inputs, new_tokens], dim=1)

                #Updating the beam scores to be used after this step in generation
                beam_scores=new_top_scores.view(-1)

                #Printing decoder_input_ids to see whats going on
                #print(f"Decoder input ids at step{step}:\n{decoder_input_ids}")

                # eos_mask = (new_tokens == tokenizer.eos_token_id) #creating 2D boolean of shape as new_tokens telling which tokens generated now are EOS by setting that value to True
                # finished_beams = finished_beams | eos_mask.view(-1) #.view(-1)Flattens eos_mask to make it a 1D tensor of shape(original_batch_size*num_beams) same as finished_beams, then we perform an OR operation(|) to update finished beam to True for the beams that have just generated EOS tokens

                # Check for early stopping
                if early_stopping:
                    # If all beams for all sequences are finished, stop decoding
                    if new_tokens[0]==tokenizer.eos_token_id: #if the top beam at the time generates the EOS token then break the loop
                        break


    # Reshape and extract the best sequence for each input
    output_sequences = decoder_input_ids.view(original_batch_size, num_beams, -1)
    best_sequences = output_sequences[:, 0, :] #creates 2D tensor of shape(original_batch_size, sequence length)

    return best_sequences


# Requires a translated list(list containing MT hypothesis) of previous iteration and index of current iteration
def constrained_translate(translated_list):
    #Tokenizing translated list

    translated_list_t = tokenizer(
                translated_list,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128
                ).to(device)
    translated_input_ids=translated_list_t["input_ids"] #This will be a 2D tensor of shape(total number of sequences tokenized, sequence length)
    translated_attention_mask=translated_list_t["attention_mask"] #This will be 2D tensor of shape(total number of sequences tokenized, sequence length)

    dataset_t = tokenizer(
                source_reference_list,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128
                ).to(device)
    source_input_ids=dataset_t["input_ids"] #This will be a 2D tensor of shape(total number of sequences tokenized, i.e. length of list passed in tokenizer, sequence length)
    source_attention_mask=dataset_t["attention_mask"] #This will be 2D tensor of shape(total number of sequences tokenized, i.e. length of list passed in tokenizer, sequence length)


    #Tokenizing target(russian) reference list
    reference_list_t = tokenizer(
            target_reference_list,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
            ).to(device)

    reference_input_ids = reference_list_t["input_ids"]  # This will be 2D tensor of shape(total number of sequences tokenized, sequence length)
    reference_attention_mask = reference_list_t["attention_mask"]  # This will be 2D tensor of shape(total number of sequences tokenized, sequence length)

    constrained_translated_list=[]


    # Iterating through each translated sequence
    for i in tqdm(range(len(translated_list))):
        translated_seq_ids = translated_input_ids[i:i+1]  # 2D tensor of shape [1, max seq_length in the tokenized list] containing token_ids for Single sequence from translated batch
        reference_seq_ids = reference_input_ids[i:i+1]  # 2D tensor of shape [1, max seq_length in the tokenized list] containing token_ids for Corresponding reference sequence
        source_seq_ids= source_input_ids[i:i+1] # 2D tensor of shape [1, max seq_length in the tokenized list] containing token_ids for Corresponding reference sequence
        source_seq_attention_mask=source_attention_mask[i:i+1]


        # Find phrases(subsequences) missing from translated_seq to add them as constraints
        constraint, position = find_missing_tokens(translated_seq_ids[0], reference_seq_ids[0])#constraint will be a 1D array of token_ids, Sending in values of the one row in translated_seq_ids and reference_seq_ids in find_missing_tokens, so these are 1D tensors(imagine like a list of integers)
                                                                                    #it could also return None in case of no mismatch

        #In case there is no mismatch in translated and reference list just add the sequence from translated_list in the constrained_translated_list, and continue to next sequence
        if constraint==None:
            constrained_translated_list.append(translated_list[i]) #.append will add the string as a single element to constrained_translated_list, if we use .extend here it will treat the string as an iterable and add each of its elements(characters) to the constrained_translated_list

            continue

        constraint_info={"token_ids":constraint, "position":position}
        #Add the constraint to the list of constraints for the current sequence
        GBS_constraints_2[i].append(constraint_info)


        #Generated translation using Grid_beam_search
        constrained_seq_ids = constrained_beam_search(
            input_ids=source_seq_ids,#sending input_ids of one sequence
            attention_mask=source_seq_attention_mask, #sending attention mask of 1 sequence
            constraints=GBS_constraints_2[i], #Sending constraints for that one sequence
        )


        constrained_translation = tokenizer.batch_decode(constrained_seq_ids, skip_special_tokens=True)
        constrained_translated_list.extend(constrained_translation) #we use .extend here because it will add each element of the iterable constrained_translation to constrained_translated_list, constrained_translation could be a list of strings, but in this case is just a list containing one string


    return constrained_translated_list


#Getting the translation of input sequence by deriving constraints for each sequence of target_reference_list subsequences that are missing from model_generated_list
constrained_translation_2=constrained_translate(constrained_translation_1)  #list of Constrained Translated Sequnces in Target Language


#BLEU score for second iteration
Base_blue_score= sacrebleu.corpus_bleu(model_generated_list, reference_list_BLEU)
print(f"\nBase Bleu score:{Base_blue_score} ")
#Confirm number of elements in hypotheses list is equal to number of sequences in reference list)
if len(constrained_translation_2)==len(reference_list_BLEU[0]):
    constrained_bleu_score2= sacrebleu.corpus_bleu(constrained_translation_2, reference_list_BLEU)
    print(f"\n Constrained Bleu Score for 2nd iteration:{constrained_bleu_score2}")

else:
    print("Mismatch in number of hypotheses and reference sequences")


100%|██████████| 2818/2818 [22:29<00:00,  2.09it/s]



Base Bleu score:BLEU = 27.48 57.4/33.3/21.3/14.0 (BP = 1.000 ratio = 1.012 hyp_len = 56568 ref_len = 55920) 

 Constrained Bleu Score:BLEU = 26.60 57.2/32.5/20.5/13.2 (BP = 1.000 ratio = 1.010 hyp_len = 56485 ref_len = 55920)


In [ ]:
## Implementing Lexical Constraints (Third iteration)
eval_dataset=dataset["validation"]
source_reference_list=[row["en"] for row in eval_dataset["translation"]] #A list of all english sentences in the dataset
target_reference_list=[row["ru"] for row in eval_dataset["translation"]] #A list of all russian sentence in the dataset
reference_list_BLEU=[[row["ru"] for row in eval_dataset["translation"]]]
generation_batch_size=1 #increase or decrease as per memory available in models processing unit
model.eval()# Putting the model in eval mode
GBS_constraints_3=GBS_constraints_2
"""
find_missing_tokens function returns the n-gram(subsequence) from reference_seq which is missing from translated_sequence,
It looks for 3-grams, if none found then for 2-grams, if none found then 1-grams, and return None if not even 1-grams are found to be missing from translated_sequence
"""
def find_missing_tokens(translated_seq, reference_seq):
    #This nested function returns a boolean value after checking if the subsequence is missing from the translated_seq
    def is_subsequence_missing(subsequence, translated_seq, n):
        #Goes over all n-grams(subsequences) in the translated_seq
        for i in range(len(translated_seq) - n + 1):
            #compares all n-grams in translated_seq and
            if torch.equal(translated_seq[i:i + n], subsequence): #if one of the subsequences in translated_seq is equal to subsequence it returns false because it means subsequence is not missing from translated_seq
                return False
        return True

    # Check for missing subsequences of in the order 3, 2, and 1 tokens
    for n in range(3, 0, -1):
        #Goes over all n-grams(subsequences) in the reference_seq
        for i in range(len(reference_seq) - n + 1): #number of n-grams in a sequence=len_of_seq-n+1, here len(reference_seq) give the length of the reference sequence since it is a 1D tensor
            subsequence = reference_seq[i:i + n] #choosing the current n-gram
            if is_subsequence_missing(subsequence, translated_seq, n): #returns True if current n-gram is missing
                return subsequence, i #basically returns the first missing n-gram we can find in the order of looking for 3,2,1-grams and the position of the first token of the n-gram in reference

    # If no missing subsequences are found, return None
    return None, None


def constrained_beam_search(input_ids, attention_mask, constraints, num_beams=4, max_length=128, early_stopping=True):
    original_batch_size = input_ids.shape[0] #will be 1, because input_ids here is a 1D tensor in case we are only sending in 1 sequence from constrained_translate() to be translated in constrained_beam_search_ at a time.

    #scaling original input to num_beams for beam search
    input_ids = input_ids.repeat_interleave(num_beams, dim=0) #creating num_beam copies the each input sequence(one for each beam).
    attention_mask = attention_mask.repeat_interleave(num_beams, dim=0) #doing the same thing again but for attention masks

    # Creating tensor for storing outputs of current beams
    decoder_input_ids = torch.full(  #making a 2D tensor of shape (original_batch_size * num_beams, 1) with each element equal to "BOS" token_id of the decoder/tokenizer
                        (original_batch_size * num_beams, 1),
                        model.config.decoder_start_token_id,
                        dtype=torch.long,
                        device=device
                        )

    # making a 1D tensor for storing scores of all beams of all sequences
    beam_scores = torch.zeros(original_batch_size * num_beams, dtype=torch.float, device=device)

    #Maintains a list of states for each beam
    constraint_states = [[{"active": True, "satisfied": False, "tokens_left": constraint_info["token_ids"].clone(), "position": constraint_info["position"]} for constraint_info in constraints] for _ in range(original_batch_size * num_beams)] #list for each beam of all input sequences, each beam(row) has a list of constraints states
                                                                                                                             #each constraint state is a dictionary with 3 keys- "active"(meaning it is actively looking to be fullfilled), "satisfied"(meaning it is satisfied), "tokens_left"(which contain a copy of the tokens to be included for that constraint in that beam)
    Start=True ##initial Condition to be checked only once during the first forward pass                                                                                                                           #Active=True means the full constraint is pending or part of the constraint is pending
    #Helper function for adding constraint token
    def find_variable_in_constraints(step, constraints):
        position_list=[]
        for i, constraint_dict in enumerate(constraints):
            if step == constraint_dict["position"] and constraint_dict["active"]==True: #Constraint must be active constraint
                position_list.append(i)  # Return the index of the dictionary whose constraint position is equal to current decoding step
        if position_list: #if their is anything in the position list
            return position_list
        return None     #Else return None
    #Decoding Steps for generating the output
    for step in range(max_length):#each decoding step, adding one token to a beam of decoder_input_ids

        #Adding Constraint
        constraint_num=find_variable_in_constraints(step, constraint_states[0]) #checking for position in first beam since all beams are updated together anyways, it will return constraints whose "position" is equal to the current step
        if constraint_num is not None: #this step is equal to position of some constraint
            #add constraint token to decoder_input_id
            constraint_token_tensor=torch.full((original_batch_size * num_beams, 1), constraint_states[0][constraint_num[0]]["tokens_left"][0],dtype=torch.long, device=device)
            """
            Explaination of above step:
            constraint_states[0] gives us a list of dictionaries containing constraint info for all constraints of the first beam(since all constraints for all beams are updated together we can take any one, it doesnt matter)
            then from the constraint_num list that has a list of indices for all constraints that have first token valid for this step, we choose the first one i.e.constraint_num[0](it wouldnt matter if you choose the second one too since the token for a specific position would be the same as they are all derived from the same reference translation)
            Then we take the first token of that constraint and prepare a tensor to add it to the decoder input_ids
            """
            #add the 1 token of constraint to beams of decoder_input_id
            decoder_input_ids=torch.cat([decoder_input_ids, constraint_token_tensor], dim=1)


            #update constraint_state for beams of the sequence
            for i in range(original_batch_size*num_beams): #for all beams
                for x in range(len(constraint_num)):  #for all constraints with position same as this step
                    constraint_states[i][constraint_num[x]]["position"]+=1 #shift position 1 index forward for the next token
                    constraint_states[i][constraint_num[x]]["tokens_left"]=constraint_states[i][constraint_num[x]]["tokens_left"][1:] #remove the token from the constraint that has been added to the decoder_input_id
                    if len(constraint_states[i][constraint_num[x]]["tokens_left"])==0: #when there are no more tokens left in that constraint
                        constraint_states[i][constraint_num[x]]["active"]=False #change the state of the constraint



        #Generating token by use of logits and Scratch beam search
        else: #this step is not a position of some constraint then we just generate using models logits
            with torch.no_grad():

                # Forward pass
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    decoder_input_ids=decoder_input_ids
                    ) #This will be a dictionary like object containing a lot of things like hidden states,etc, but we are most interested in logits
                """
                Talking about some important Tensor Operations:

                Broadcasting:
                In pytorch, a process called broadcasting happens just before an element wise operation that requires two tensors with same number of dimensions to align in shape.
                Example Adding tensor A of shape(4,6,8) with tensor B of shape(4,1,1) then the scaler values in tensor B along the dimensions with only 1 element are duplicated to match the shape of the tensor A.
                But broadcasting only happens when two tensors have same size of some dimensions and some dimensions of size 1.

                tensor.view(size of dim=0, size of dim=1.....) is used to create a reshaped tensor with the same number of elements as the original tensor.
                It can add new dimensions of a specific size as long as the number of element in the original tensor could be accomodated.
                tensor.view(-1,10) tell us to accomodate 10 elements in dim=10 and infer what number of element to accomodate in dim=0 such that the total number of elements remain the same.

                tensor.unsqueeze() is used to add a singleton dimension(dimension of size 1) at a specified index
                tensor.unsequeeze(1) adds a singleton dimension at index=1

                tensor.squeeze() is used remove a singleton dimension from a specified index

                tensor.expand() is used to basically perform broadcasting on command, you would send in sizes of dimension of tensor which are capable of broadcasting and get a result.
                This means that the size you send in should either be the same as sizes of dimension of original tensor OR in case of singleton dimension in original tensor you could send in a greater number to stretch along that dimension
                tensor.expand(-1, 4) tells to keep dim=0 as is and basically stretch dim=1 to size 4, this is conditioned on the fact that the dimension to be stretched is of size 1 otherwise an error will occur

                tensor.expand_as(target_tensor) is used to perform broadcasting on tensor conditioned on the fact that the size of target_tensor is suitable for broadcasting


                """
                # Creating a 2D tensor Logits
                logits = outputs.logits[:, -1, :] #output.logits is a 3-D tensor of shape(original_batch_size*num_beam, current sequence length in decoder_input_ids, vocab size). where each element is the logit of each token_id of each sequence for each element of the vocabulary
                                                #By slicing we choose the last token_id of each sequence it to get a 2-D tensor of shape(original_batch_size*num_beam, vocab_size) where each element is a logit of last token_id of each sequence element for each element of the vocabulary
                #print(f"Logits at step {step}:\n{logits}")
                # Using Boolean masking to mark finished beams (those that already generated EOS)
                #finished_mask = finished_beams.view(-1, 1).expand(-1, logits.size(-1))  # finished_mask will be a 2D of shape(original_batch_size * num_beams, vocab_size) with values of True/False repeated in each row(representing each beam) upto the vocab_size
                #logits[finished_mask] = -float('inf')  # Making logits of lasts token for all elements of vocabulary for the finished beams negative infinity, By sending in boolean mask tensor of the same shape as the logits tensor the operation is applied to all logits elements that correspond with True in finished mask.
                                                        #Making Logit for all words in vocabulary negative infinity means probability=0 after softmax, and log of that is a big negative value, when that big negative value is combined with beam_score to get next_score it ensures that these beams will now not come in the topk.
                                                        #This ensures that only active beams (those not yet finished) are considered during the top-k selection.

                # Creating 2D tensor log probabilities
                log_probs = F.log_softmax(logits, dim=-1) #performs softmax on all logits normalizing them to probability of 0-1, then takes log of that so the highest probability is turend into the largest number(smallest negative value).
                                                        #dim=-1 takes softmax along the last dimension(2nd dimension with size=vocab_size in this case) which means that normalization happens for each row(last token for each sequence of decoder_input_ids)
                #print(f"Log_probs at step {step}:\n{log_probs}")
                #get top num_beams scores from each beam
                total_top_log_probs, total_top_token_ids=torch.topk(log_probs, num_beams, dim=1) #give 2 tensors of shape(original_batch_size*num_beams, num_beams) where each row contains scores top num_beam scores for that row and the corresponding token_ids in descending
                #print(f"Topk log_probs at step {step}:\n{total_top_log_probs}")
                #print(f"Topk token_ids at step {step}:\n{total_top_token_ids}")

                if Start==True:
                    new_tokens=total_top_token_ids[::num_beams].reshape(-1,1) #using slicing with step value num_beam we take the top 4 tokens of a beam of a sequence and make a tensor containing new tokens for each beam, .reshape is used in place of .view() becauce we used slicing and so tensor is not contiguous in memory hence .view wont work
                    decoder_input_ids=torch.cat([decoder_input_ids[torch.arange(original_batch_size*num_beams)],new_tokens], dim=1) #adding first 4 tokens to different beam of each sequence
                    beam_scores=total_top_log_probs[::num_beams].reshape(-1) #slicing to make tensor of shape(original_batch_size,num_beams) and then we flatten it using .reshape(-1)
                    Start=False
                    #print(f"Decoder_input_ids at step{step}:\n{decoder_input_ids}")
                    #print(f"Beam score at step{step}:\n{beam_scores}")

                    continue

                #calculate total scores for each beams top 4 tokens
                total_top_score=total_top_log_probs+beam_scores.unsqueeze(1)
                #print(f"Total score of each topk token of each beam scores at step {step}:\n{total_top_score}")


                #reshape that
                reshaped_total_top_score=total_top_score.view(original_batch_size, num_beams*num_beams)

                #Getting the best scores for this step in generation
                new_top_scores,new_token_indices=torch.topk(reshaped_total_top_score,num_beams, dim=1) #gives 2 tensors of shape (original_batch_size, num_beam) containing the top num_beam score from all total_scores of one sequence in each row in descending order and their index in that row in total_top_score_reshaped
                #print(f"Total Top scores for new tokens of each sequence scores at step {step}:\n{new_top_scores}")
                #calculating beam indices for the new tokens of each sequence with best score
                beam_indices=new_token_indices//num_beams #elements of a row in this tensor will tell which token originally belonged to which beam for a sequence
                #reshaping beam_indices and scaling indices of elements from row level to global level in the tensor
                base_indices=torch.arange(original_batch_size,device=device).unsqueeze(1) * num_beams
                scaled_beam_indices=beam_indices+base_indices #Tensor of shape shape (original_batch_size, num_beam)
                flattened_scaled_beam_indices=scaled_beam_indices.view(-1) #making a 1D tensor of shape(original_batch_size*num_beam)

                #By Advance Indexing using these scaled indices we will obtain sequences from decoder_input_ids of the respective beam that the indices point to
                prev_decoder_inputs=decoder_input_ids[flattened_scaled_beam_indices]

                #Getting the token_ids of the tokens to be added
                reshaped_top_total_token_ids=total_top_token_ids.view(original_batch_size, num_beams*num_beams)
                """
                Here each row will contain the token ids of the top scoring tokens of all beams of a sequence just like the structure of reshaped_total_top_score
                Since new_token_indices tells the row index of the topk elements of that row, the same indices will also point to the token_ids in reshaped_top_total_token_ids
                So we make a vector of shape (original_batch_size,1) where each element will help us point to a row indices in reshaped_top_total_token_ids
                And then new_token_indices has column indices which help us point to columns of reshaped_top_total_token_ids for that row
                in this case of Advacne indexing because we use two rows and columns we get a new tensor of shape of rows and columns used
                """

                raw_new_tokens = reshaped_top_total_token_ids[torch.arange(original_batch_size,device=device).unsqueeze(1), new_token_indices] #the token_ids for the new tokens in a Tensor of shape(original_batch_size, num_beams)
                new_tokens = raw_new_tokens.view(original_batch_size*num_beams,1) #2D Tensor of shape(original_batch_size*num_beam,1) made to be suited for concatenation to decoder_input_ids
                decoder_input_ids=torch.cat([prev_decoder_inputs, new_tokens], dim=1)

                #Updating the beam scores to be used after this step in generation
                beam_scores=new_top_scores.view(-1)

                #Printing decoder_input_ids to see whats going on
                #print(f"Decoder input ids at step{step}:\n{decoder_input_ids}")

                # eos_mask = (new_tokens == tokenizer.eos_token_id) #creating 2D boolean of shape as new_tokens telling which tokens generated now are EOS by setting that value to True
                # finished_beams = finished_beams | eos_mask.view(-1) #.view(-1)Flattens eos_mask to make it a 1D tensor of shape(original_batch_size*num_beams) same as finished_beams, then we perform an OR operation(|) to update finished beam to True for the beams that have just generated EOS tokens

                # Check for early stopping
                if early_stopping:
                    # If all beams for all sequences are finished, stop decoding
                    if new_tokens[0]==tokenizer.eos_token_id: #if the top beam at the time generates the EOS token then break the loop
                        break


    # Reshape and extract the best sequence for each input
    output_sequences = decoder_input_ids.view(original_batch_size, num_beams, -1)
    best_sequences = output_sequences[:, 0, :] #creates 2D tensor of shape(original_batch_size, sequence length)

    return best_sequences


# Requires a translated list(list containing MT hypothesis) of previous iteration and index of current iteration
def constrained_translate(translated_list):
    #Tokenizing translated list

    translated_list_t = tokenizer(
                translated_list,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128
                ).to(device)
    translated_input_ids=translated_list_t["input_ids"] #This will be a 2D tensor of shape(total number of sequences tokenized, sequence length)
    translated_attention_mask=translated_list_t["attention_mask"] #This will be 2D tensor of shape(total number of sequences tokenized, sequence length)

    dataset_t = tokenizer(
                source_reference_list,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128
                ).to(device)
    source_input_ids=dataset_t["input_ids"] #This will be a 2D tensor of shape(total number of sequences tokenized, i.e. length of list passed in tokenizer, sequence length)
    source_attention_mask=dataset_t["attention_mask"] #This will be 2D tensor of shape(total number of sequences tokenized, i.e. length of list passed in tokenizer, sequence length)


    #Tokenizing target(russian) reference list
    reference_list_t = tokenizer(
            target_reference_list,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
            ).to(device)

    reference_input_ids = reference_list_t["input_ids"]  # This will be 2D tensor of shape(total number of sequences tokenized, sequence length)
    reference_attention_mask = reference_list_t["attention_mask"]  # This will be 2D tensor of shape(total number of sequences tokenized, sequence length)

    constrained_translated_list=[]


    # Iterating through each translated sequence
    for i in tqdm(range(len(translated_list))):
        translated_seq_ids = translated_input_ids[i:i+1]  # 2D tensor of shape [1, max seq_length in the tokenized list] containing token_ids for Single sequence from translated batch
        reference_seq_ids = reference_input_ids[i:i+1]  # 2D tensor of shape [1, max seq_length in the tokenized list] containing token_ids for Corresponding reference sequence
        source_seq_ids= source_input_ids[i:i+1] # 2D tensor of shape [1, max seq_length in the tokenized list] containing token_ids for Corresponding reference sequence
        source_seq_attention_mask=source_attention_mask[i:i+1]


        # Find phrases(subsequences) missing from translated_seq to add them as constraints
        constraint, position = find_missing_tokens(translated_seq_ids[0], reference_seq_ids[0])#constraint will be a 1D array of token_ids, Sending in values of the one row in translated_seq_ids and reference_seq_ids in find_missing_tokens, so these are 1D tensors(imagine like a list of integers)
                                                                                    #it could also return None in case of no mismatch

        #In case there is no mismatch in translated and reference list just add the sequence from translated_list in the constrained_translated_list, and continue to next sequence
        if constraint==None:
            constrained_translated_list.append(translated_list[i]) #.append will add the string as a single element to constrained_translated_list, if we use .extend here it will treat the string as an iterable and add each of its elements(characters) to the constrained_translated_list
            continue

        constraint_info={"token_ids":constraint, "position":position}
        #Add the constraint to the list of constraints for the current sequence
        GBS_constraints_3[i].append(constraint_info)


        #Generated translation using Grid_beam_search
        constrained_seq_ids = constrained_beam_search(
            input_ids=source_seq_ids,#sending input_ids of one sequence
            attention_mask=source_seq_attention_mask, #sending attention mask of 1 sequence
            constraints=GBS_constraints_3[i], #Sending constraints for that one sequence
        )


        constrained_translation = tokenizer.batch_decode(constrained_seq_ids, skip_special_tokens=True)
        constrained_translated_list.extend(constrained_translation) #we use .extend here because it will add each element of the iterable constrained_translation to constrained_translated_list, constrained_translation could be a list of strings, but in this case is just a list containing one string

    return constrained_translated_list


#Getting the translation of input sequence by deriving constraints for each sequence of target_reference_list subsequences that are missing from model_generated_list
constrained_translation_3=constrained_translate(constrained_translation_2)  #list of Constrained Translated Sequnces in Target Language


#BLEU score for Third iteration
Base_blue_score= sacrebleu.corpus_bleu(model_generated_list, reference_list_BLEU)
print(f"\nBase Bleu score:{Base_blue_score} ")
#Confirm number of elements in hypotheses list is equal to number of sequences in reference list)
if len(constrained_translation_3)==len(reference_list_BLEU[0]):
    constrained_bleu_score3= sacrebleu.corpus_bleu(constrained_translation_3, reference_list_BLEU)
    print(f"\n Constrained Bleu Score for 3rd iteration:{constrained_bleu_score3}")

else:
    print("Mismatch in number of hypotheses and reference sequences")


100%|██████████| 2818/2818 [22:28<00:00,  2.09it/s]



Base Bleu score:BLEU = 27.48 57.4/33.3/21.3/14.0 (BP = 1.000 ratio = 1.012 hyp_len = 56568 ref_len = 55920) 

 Constrained Bleu Score:BLEU = 26.47 57.1/32.3/20.4/13.1 (BP = 1.000 ratio = 1.009 hyp_len = 56443 ref_len = 55920)


In [ ]:
## Implementing Lexical Constraints (Fourth iteration)
eval_dataset=dataset["validation"]
source_reference_list=[row["en"] for row in eval_dataset["translation"]] #A list of all english sentences in the dataset
target_reference_list=[row["ru"] for row in eval_dataset["translation"]] #A list of all russian sentence in the dataset
reference_list_BLEU=[[row["ru"] for row in eval_dataset["translation"]]]
generation_batch_size=1 #increase or decrease as per memory available in models processing unit
model.eval()# Putting the model in eval mode
GBS_constraints_4=GBS_constraints_3
"""
find_missing_tokens function returns the n-gram(subsequence) from reference_seq which is missing from translated_sequence,
It looks for 3-grams, if none found then for 2-grams, if none found then 1-grams, and return None if not even 1-grams are found to be missing from translated_sequence
"""
def find_missing_tokens(translated_seq, reference_seq):
    #This nested function returns a boolean value after checking if the subsequence is missing from the translated_seq
    def is_subsequence_missing(subsequence, translated_seq, n):
        #Goes over all n-grams(subsequences) in the translated_seq
        for i in range(len(translated_seq) - n + 1):
            #compares all n-grams in translated_seq and
            if torch.equal(translated_seq[i:i + n], subsequence): #if one of the subsequences in translated_seq is equal to subsequence it returns false because it means subsequence is not missing from translated_seq
                return False
        return True

    # Check for missing subsequences of in the order 3, 2, and 1 tokens
    for n in range(3, 0, -1):
        #Goes over all n-grams(subsequences) in the reference_seq
        for i in range(len(reference_seq) - n + 1): #number of n-grams in a sequence=len_of_seq-n+1, here len(reference_seq) give the length of the reference sequence since it is a 1D tensor
            subsequence = reference_seq[i:i + n] #choosing the current n-gram
            if is_subsequence_missing(subsequence, translated_seq, n): #returns True if current n-gram is missing
                return subsequence, i #basically returns the first missing n-gram we can find in the order of looking for 3,2,1-grams and the position of the first token of the n-gram in reference

    # If no missing subsequences are found, return None
    return None, None


def constrained_beam_search(input_ids, attention_mask, constraints, num_beams=4, max_length=128, early_stopping=True):
    original_batch_size = input_ids.shape[0] #will be 1, because input_ids here is a 1D tensor in case we are only sending in 1 sequence from constrained_translate() to be translated in constrained_beam_search_ at a time.

    #scaling original input to num_beams for beam search
    input_ids = input_ids.repeat_interleave(num_beams, dim=0) #creating num_beam copies the each input sequence(one for each beam).
    attention_mask = attention_mask.repeat_interleave(num_beams, dim=0) #doing the same thing again but for attention masks

    # Creating tensor for storing outputs of current beams
    decoder_input_ids = torch.full(  #making a 2D tensor of shape (original_batch_size * num_beams, 1) with each element equal to "BOS" token_id of the decoder/tokenizer
                        (original_batch_size * num_beams, 1),
                        model.config.decoder_start_token_id,
                        dtype=torch.long,
                        device=device
                        )

    # making a 1D tensor for storing scores of all beams of all sequences
    beam_scores = torch.zeros(original_batch_size * num_beams, dtype=torch.float, device=device)

    #Maintains a list of states for each beam
    constraint_states = [[{"active": True, "satisfied": False, "tokens_left": constraint_info["token_ids"].clone(), "position": constraint_info["position"]} for constraint_info in constraints] for _ in range(original_batch_size * num_beams)] #list for each beam of all input sequences, each beam(row) has a list of constraints states
                                                                                                                             #each constraint state is a dictionary with 3 keys- "active"(meaning it is actively looking to be fullfilled), "satisfied"(meaning it is satisfied), "tokens_left"(which contain a copy of the tokens to be included for that constraint in that beam)
    Start=True ##initial Condition to be checked only once during the first forward pass                                                                                                                           #Active=True means the full constraint is pending or part of the constraint is pending
    #Helper function for adding constraint token
    def find_variable_in_constraints(step, constraints):
        position_list=[]
        for i, constraint_dict in enumerate(constraints):
            if step == constraint_dict["position"] and constraint_dict["active"]==True: #Constraint must be active constraint
                position_list.append(i)  # Return the index of the dictionary whose constraint position is equal to current decoding step
        if position_list: #if their is anything in the position list
            return position_list
        return None     #Else return None
    #Decoding Steps for generating the output
    for step in range(max_length):#each decoding step, adding one token to a beam of decoder_input_ids

        #Adding Constraint
        constraint_num=find_variable_in_constraints(step, constraint_states[0]) #checking for position in first beam since all beams are updated together anyways, it will return constraints whose "position" is equal to the current step
        if constraint_num is not None: #this step is equal to position of some constraint
            #add constraint token to decoder_input_id
            constraint_token_tensor=torch.full((original_batch_size * num_beams, 1), constraint_states[0][constraint_num[0]]["tokens_left"][0],dtype=torch.long, device=device)
            """
            Explaination of above step:
            constraint_states[0] gives us a list of dictionaries containing constraint info for all constraints of the first beam(since all constraints for all beams are updated together we can take any one, it doesnt matter)
            then from the constraint_num list that has a list of indices for all constraints that have first token valid for this step, we choose the first one i.e.constraint_num[0](it wouldnt matter if you choose the second one too since the token for a specific position would be the same as they are all derived from the same reference translation)
            Then we take the first token of that constraint and prepare a tensor to add it to the decoder input_ids
            """
            #add the 1 token of constraint to beams of decoder_input_id
            decoder_input_ids=torch.cat([decoder_input_ids, constraint_token_tensor], dim=1)


            #update constraint_state for beams of the sequence
            for i in range(original_batch_size*num_beams): #for all beams
                for x in range(len(constraint_num)):  #for all constraints with position same as this step
                    constraint_states[i][constraint_num[x]]["position"]+=1 #shift position 1 index forward for the next token
                    constraint_states[i][constraint_num[x]]["tokens_left"]=constraint_states[i][constraint_num[x]]["tokens_left"][1:] #remove the token from the constraint that has been added to the decoder_input_id
                    if len(constraint_states[i][constraint_num[x]]["tokens_left"])==0: #when there are no more tokens left in that constraint
                        constraint_states[i][constraint_num[x]]["active"]=False #change the state of the constraint



        #Generating token by use of logits and Scratch beam search
        else: #this step is not a position of some constraint then we just generate using models logits
            with torch.no_grad():

                # Forward pass
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    decoder_input_ids=decoder_input_ids
                    ) #This will be a dictionary like object containing a lot of things like hidden states,etc, but we are most interested in logits
                """
                Talking about some important Tensor Operations:

                Broadcasting:
                In pytorch, a process called broadcasting happens just before an element wise operation that requires two tensors with same number of dimensions to align in shape.
                Example Adding tensor A of shape(4,6,8) with tensor B of shape(4,1,1) then the scaler values in tensor B along the dimensions with only 1 element are duplicated to match the shape of the tensor A.
                But broadcasting only happens when two tensors have same size of some dimensions and some dimensions of size 1.

                tensor.view(size of dim=0, size of dim=1.....) is used to create a reshaped tensor with the same number of elements as the original tensor.
                It can add new dimensions of a specific size as long as the number of element in the original tensor could be accomodated.
                tensor.view(-1,10) tell us to accomodate 10 elements in dim=10 and infer what number of element to accomodate in dim=0 such that the total number of elements remain the same.

                tensor.unsqueeze() is used to add a singleton dimension(dimension of size 1) at a specified index
                tensor.unsequeeze(1) adds a singleton dimension at index=1

                tensor.squeeze() is used remove a singleton dimension from a specified index

                tensor.expand() is used to basically perform broadcasting on command, you would send in sizes of dimension of tensor which are capable of broadcasting and get a result.
                This means that the size you send in should either be the same as sizes of dimension of original tensor OR in case of singleton dimension in original tensor you could send in a greater number to stretch along that dimension
                tensor.expand(-1, 4) tells to keep dim=0 as is and basically stretch dim=1 to size 4, this is conditioned on the fact that the dimension to be stretched is of size 1 otherwise an error will occur

                tensor.expand_as(target_tensor) is used to perform broadcasting on tensor conditioned on the fact that the size of target_tensor is suitable for broadcasting


                """
                # Creating a 2D tensor Logits
                logits = outputs.logits[:, -1, :] #output.logits is a 3-D tensor of shape(original_batch_size*num_beam, current sequence length in decoder_input_ids, vocab size). where each element is the logit of each token_id of each sequence for each element of the vocabulary
                                                #By slicing we choose the last token_id of each sequence it to get a 2-D tensor of shape(original_batch_size*num_beam, vocab_size) where each element is a logit of last token_id of each sequence element for each element of the vocabulary
                #print(f"Logits at step {step}:\n{logits}")
                # Using Boolean masking to mark finished beams (those that already generated EOS)
                #finished_mask = finished_beams.view(-1, 1).expand(-1, logits.size(-1))  # finished_mask will be a 2D of shape(original_batch_size * num_beams, vocab_size) with values of True/False repeated in each row(representing each beam) upto the vocab_size
                #logits[finished_mask] = -float('inf')  # Making logits of lasts token for all elements of vocabulary for the finished beams negative infinity, By sending in boolean mask tensor of the same shape as the logits tensor the operation is applied to all logits elements that correspond with True in finished mask.
                                                        #Making Logit for all words in vocabulary negative infinity means probability=0 after softmax, and log of that is a big negative value, when that big negative value is combined with beam_score to get next_score it ensures that these beams will now not come in the topk.
                                                        #This ensures that only active beams (those not yet finished) are considered during the top-k selection.

                # Creating 2D tensor log probabilities
                log_probs = F.log_softmax(logits, dim=-1) #performs softmax on all logits normalizing them to probability of 0-1, then takes log of that so the highest probability is turend into the largest number(smallest negative value).
                                                        #dim=-1 takes softmax along the last dimension(2nd dimension with size=vocab_size in this case) which means that normalization happens for each row(last token for each sequence of decoder_input_ids)
                #print(f"Log_probs at step {step}:\n{log_probs}")
                #get top num_beams scores from each beam
                total_top_log_probs, total_top_token_ids=torch.topk(log_probs, num_beams, dim=1) #give 2 tensors of shape(original_batch_size*num_beams, num_beams) where each row contains scores top num_beam scores for that row and the corresponding token_ids in descending
                #print(f"Topk log_probs at step {step}:\n{total_top_log_probs}")
                #print(f"Topk token_ids at step {step}:\n{total_top_token_ids}")

                if Start==True:
                    new_tokens=total_top_token_ids[::num_beams].reshape(-1,1) #using slicing with step value num_beam we take the top 4 tokens of a beam of a sequence and make a tensor containing new tokens for each beam, .reshape is used in place of .view() becauce we used slicing and so tensor is not contiguous in memory hence .view wont work
                    decoder_input_ids=torch.cat([decoder_input_ids[torch.arange(original_batch_size*num_beams)],new_tokens], dim=1) #adding first 4 tokens to different beam of each sequence
                    beam_scores=total_top_log_probs[::num_beams].reshape(-1) #slicing to make tensor of shape(original_batch_size,num_beams) and then we flatten it using .reshape(-1)
                    Start=False
                    #print(f"Decoder_input_ids at step{step}:\n{decoder_input_ids}")
                    #print(f"Beam score at step{step}:\n{beam_scores}")

                    continue

                #calculate total scores for each beams top 4 tokens
                total_top_score=total_top_log_probs+beam_scores.unsqueeze(1)
                #print(f"Total score of each topk token of each beam scores at step {step}:\n{total_top_score}")


                #reshape that
                reshaped_total_top_score=total_top_score.view(original_batch_size, num_beams*num_beams)

                #Getting the best scores for this step in generation
                new_top_scores,new_token_indices=torch.topk(reshaped_total_top_score,num_beams, dim=1) #gives 2 tensors of shape (original_batch_size, num_beam) containing the top num_beam score from all total_scores of one sequence in each row in descending order and their index in that row in total_top_score_reshaped
                #print(f"Total Top scores for new tokens of each sequence scores at step {step}:\n{new_top_scores}")
                #calculating beam indices for the new tokens of each sequence with best score
                beam_indices=new_token_indices//num_beams #elements of a row in this tensor will tell which token originally belonged to which beam for a sequence
                #reshaping beam_indices and scaling indices of elements from row level to global level in the tensor
                base_indices=torch.arange(original_batch_size,device=device).unsqueeze(1) * num_beams
                scaled_beam_indices=beam_indices+base_indices #Tensor of shape shape (original_batch_size, num_beam)
                flattened_scaled_beam_indices=scaled_beam_indices.view(-1) #making a 1D tensor of shape(original_batch_size*num_beam)

                #By Advance Indexing using these scaled indices we will obtain sequences from decoder_input_ids of the respective beam that the indices point to
                prev_decoder_inputs=decoder_input_ids[flattened_scaled_beam_indices]

                #Getting the token_ids of the tokens to be added
                reshaped_top_total_token_ids=total_top_token_ids.view(original_batch_size, num_beams*num_beams)
                """
                Here each row will contain the token ids of the top scoring tokens of all beams of a sequence just like the structure of reshaped_total_top_score
                Since new_token_indices tells the row index of the topk elements of that row, the same indices will also point to the token_ids in reshaped_top_total_token_ids
                So we make a vector of shape (original_batch_size,1) where each element will help us point to a row indices in reshaped_top_total_token_ids
                And then new_token_indices has column indices which help us point to columns of reshaped_top_total_token_ids for that row
                in this case of Advacne indexing because we use two rows and columns we get a new tensor of shape of rows and columns used
                """

                raw_new_tokens = reshaped_top_total_token_ids[torch.arange(original_batch_size,device=device).unsqueeze(1), new_token_indices] #the token_ids for the new tokens in a Tensor of shape(original_batch_size, num_beams)
                new_tokens = raw_new_tokens.view(original_batch_size*num_beams,1) #2D Tensor of shape(original_batch_size*num_beam,1) made to be suited for concatenation to decoder_input_ids
                decoder_input_ids=torch.cat([prev_decoder_inputs, new_tokens], dim=1)

                #Updating the beam scores to be used after this step in generation
                beam_scores=new_top_scores.view(-1)

                #Printing decoder_input_ids to see whats going on
                #print(f"Decoder input ids at step{step}:\n{decoder_input_ids}")

                # eos_mask = (new_tokens == tokenizer.eos_token_id) #creating 2D boolean of shape as new_tokens telling which tokens generated now are EOS by setting that value to True
                # finished_beams = finished_beams | eos_mask.view(-1) #.view(-1)Flattens eos_mask to make it a 1D tensor of shape(original_batch_size*num_beams) same as finished_beams, then we perform an OR operation(|) to update finished beam to True for the beams that have just generated EOS tokens

                # Check for early stopping
                if early_stopping:
                    # If all beams for all sequences are finished, stop decoding
                    if new_tokens[0]==tokenizer.eos_token_id: #if the top beam at the time generates the EOS token then break the loop
                        break


    # Reshape and extract the best sequence for each input
    output_sequences = decoder_input_ids.view(original_batch_size, num_beams, -1)
    best_sequences = output_sequences[:, 0, :] #creates 2D tensor of shape(original_batch_size, sequence length)

    return best_sequences


# Requires a translated list(list containing MT hypothesis) of previous iteration and index of current iteration
def constrained_translate(translated_list):
    #Tokenizing translated list

    translated_list_t = tokenizer(
                translated_list,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128
                ).to(device)
    translated_input_ids=translated_list_t["input_ids"] #This will be a 2D tensor of shape(total number of sequences tokenized, sequence length)
    translated_attention_mask=translated_list_t["attention_mask"] #This will be 2D tensor of shape(total number of sequences tokenized, sequence length)

    dataset_t = tokenizer(
                source_reference_list,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128
                ).to(device)
    source_input_ids=dataset_t["input_ids"] #This will be a 2D tensor of shape(total number of sequences tokenized, i.e. length of list passed in tokenizer, sequence length)
    source_attention_mask=dataset_t["attention_mask"] #This will be 2D tensor of shape(total number of sequences tokenized, i.e. length of list passed in tokenizer, sequence length)


    #Tokenizing target(russian) reference list
    reference_list_t = tokenizer(
            target_reference_list,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
            ).to(device)

    reference_input_ids = reference_list_t["input_ids"]  # This will be 2D tensor of shape(total number of sequences tokenized, sequence length)
    reference_attention_mask = reference_list_t["attention_mask"]  # This will be 2D tensor of shape(total number of sequences tokenized, sequence length)

    constrained_translated_list=[]


    # Iterating through each translated sequence
    for i in tqdm(range(len(translated_list))):
        translated_seq_ids = translated_input_ids[i:i+1]  # 2D tensor of shape [1, max seq_length in the tokenized list] containing token_ids for Single sequence from translated batch
        reference_seq_ids = reference_input_ids[i:i+1]  # 2D tensor of shape [1, max seq_length in the tokenized list] containing token_ids for Corresponding reference sequence
        source_seq_ids= source_input_ids[i:i+1] # 2D tensor of shape [1, max seq_length in the tokenized list] containing token_ids for Corresponding reference sequence
        source_seq_attention_mask=source_attention_mask[i:i+1]


        # Find phrases(subsequences) missing from translated_seq to add them as constraints
        constraint, position = find_missing_tokens(translated_seq_ids[0], reference_seq_ids[0])#constraint will be a 1D array of token_ids, Sending in values of the one row in translated_seq_ids and reference_seq_ids in find_missing_tokens, so these are 1D tensors(imagine like a list of integers)
                                                                                    #it could also return None in case of no mismatch

        #In case there is no mismatch in translated and reference list just add the sequence from translated_list in the constrained_translated_list, and continue to next sequence
        if constraint==None:
            constrained_translated_list.append(translated_list[i]) #.append will add the string as a single element to constrained_translated_list, if we use .extend here it will treat the string as an iterable and add each of its elements(characters) to the constrained_translated_list
            continue

        constraint_info={"token_ids":constraint, "position":position}
        #Add the constraint to the list of constraints for the current sequence
        GBS_constraints_4[i].append(constraint_info)


        #Generated translation using Grid_beam_search
        constrained_seq_ids = constrained_beam_search(
            input_ids=source_seq_ids,#sending input_ids of one sequence
            attention_mask=source_seq_attention_mask, #sending attention mask of 1 sequence
            constraints=GBS_constraints_4[i], #Sending constraints for that one sequence
        )


        constrained_translation = tokenizer.batch_decode(constrained_seq_ids, skip_special_tokens=True)
        constrained_translated_list.extend(constrained_translation) #we use .extend here because it will add each element of the iterable constrained_translation to constrained_translated_list, constrained_translation could be a list of strings, but in this case is just a list containing one string

    return constrained_translated_list


#Getting the translation of input sequence by deriving constraints for each sequence of target_reference_list subsequences that are missing from model_generated_list
constrained_translation_4=constrained_translate(constrained_translation_3)  #list of Constrained Translated Sequnces in Target Language


#BLEU score for Third iteration
Base_blue_score= sacrebleu.corpus_bleu(model_generated_list, reference_list_BLEU)
print(f"\nBase Bleu score:{Base_blue_score} ")
#Confirm number of elements in hypotheses list is equal to number of sequences in reference list)
if len(constrained_translation_4)==len(reference_list_BLEU[0]):
    constrained_bleu_score4= sacrebleu.corpus_bleu(constrained_translation_4, reference_list_BLEU)
    print(f"\n Constrained Bleu Score for 4rth iteration:{constrained_bleu_score4}")

else:
    print("Mismatch in number of hypotheses and reference sequences")


100%|██████████| 2818/2818 [22:28<00:00,  2.09it/s]



Base Bleu score:BLEU = 27.48 57.4/33.3/21.3/14.0 (BP = 1.000 ratio = 1.012 hyp_len = 56568 ref_len = 55920) 

 Constrained Bleu Score:BLEU = 26.32 57.1/32.2/20.2/12.9 (BP = 1.000 ratio = 1.011 hyp_len = 56549 ref_len = 55920)


## Section 5: Trying Constrained Beam Search for single instances in the dataset

### Developer Interpretation of what is happening here:
1. We tokenize an input English sentence then use forward pass and generate condensed decoder_input_ids. These input ids could be decoded and formed a sentence. These sentences are then added to a translated sequence list and used as baseline for implementing constraints for the first time.
2. But when we tokenize that sentence using the tokenizer(setting the max_length=128) it expands and maybe even overflow thus being truncated and losing tokens at the end. Then if you try to decode that sentence you will see loss of words at the end. 
3. That said there are still many constraints that could be even if some sentences overflow and get truncated because others wont overflow and retain all information and the ones that do overflow would probably still have a constraints to be identified by comparing them to reference translations. 
4. Now we implement constraints by observing the tokenized forms of reference translations and try to identify sub-sequences up-to 3 tokens for each sequence in them, that are not present in baseline translation. 
5. For first iteration/cycle of applying constraints we choose 1 constraint for each sequence and get translation of input sequences using constrained_beam_search(). Now that we have these translations from the first translation we store them in a list.
6. For our second iteration we identify sub-sequences of up to 3 tokens in the tokenized form of reference translation that are missing from the tokenized form of sequences of translations from first iteration, these sub-sequences are our second set of constraints. We use the both set of constraints to get translation of input sequences using the constrained_beam_search(). 
7. For all subsequent iterations we could keep identifying additional sets of constraints by using reference translations and the translations of the previous iteration. 
8. As we keep applying constraints our translations naturally become more and more similar to reference translation so our BLEU score increases. 

#### Points to keep in mind when running this: 
* Obtaining the model_generated_list(the baseline translation list) will be a prerequisite for implementing constraints in this first iteration. Make sure you have that part correct before making observations
* Then just change the splicing indices at the top of this cell to any value of one element.  
* For seeing constraints of subsequent iteration just update the GBS_constraint variable

In [78]:
## Checking and confirming applied constraints one for single seqeunce

eval_dataset=dataset["validation"]
source_reference_list=[row["en"] for row in eval_dataset["translation"]][54:55] #A list of all english sentences in the dataset
target_reference_list=[row["ru"] for row in eval_dataset["translation"]][54:55] #A list of all russian sentence in the dataset
reference_list_BLEU=[[row["ru"] for row in eval_dataset["translation"][54:55]]]
generation_batch_size=1 #increase or decrease as per memory available in models processing unit
model.eval()# Putting the model in eval mode
GBS_constraints=[[] for _ in range(len(source_reference_list))] # we make a list containing len(source_reference_list) empty lists where each empty list will contain contraints for each sequence in the future This will be list of lists that contain subsequences(lexical constraints) for each input sequence in eval_dataset,

"""
find_missing_tokens function returns the n-gram(subsequence) from reference_seq which is missing from translated_sequence,
It looks for 3-grams, if none found then for 2-grams, if none found then 1-grams, and return None if not even 1-grams are found to be missing from translated_sequence
"""
def find_missing_tokens(translated_seq, reference_seq):
    #This nested function returns a boolean value after checking if the subsequence is missing from the translated_seq
    def is_subsequence_missing(subsequence, translated_seq, n):
        #Goes over all n-grams(subsequences) in the translated_seq
        for i in range(len(translated_seq) - n + 1):
            #compares all n-grams in translated_seq and
            if torch.equal(translated_seq[i:i + n], subsequence): #if one of the subsequences in translated_seq is equal to subsequence it returns false because it means subsequence is not missing from translated_seq
                return False
        return True

    # Check for missing subsequences of in the order 3, 2, and 1 tokens
    for n in range(3, 0, -1):
        #Goes over all n-grams(subsequences) in the reference_seq
        for i in range(len(reference_seq) - n + 1): #number of n-grams in a sequence=len_of_seq-n+1, here len(reference_seq) give the length of the reference sequence since it is a 1D tensor
            subsequence = reference_seq[i:i + n] #choosing the current n-gram
            if is_subsequence_missing(subsequence, translated_seq, n): #returns True if current n-gram is missing
                return subsequence, i #basically returns the first missing n-gram we can find in the order of looking for 3,2,1-grams and the position of the first token of the n-gram in reference

    # If no missing subsequences are found, return None
    return None, None

#The main function for implementing constraints
def constrained_beam_search(input_ids, attention_mask, constraints, num_beams=4, max_length=128, early_stopping=True):
    original_batch_size = input_ids.shape[0] #will be 1, because input_ids here is a 1D tensor in case we are only sending in 1 sequence from constrained_translate() to be translated in constrained_beam_search_ at a time.

    #scaling original input to num_beams for beam search
    input_ids = input_ids.repeat_interleave(num_beams, dim=0) #creating num_beam copies the each input sequence(one for each beam).
    attention_mask = attention_mask.repeat_interleave(num_beams, dim=0) #doing the same thing again but for attention masks

    # Creating tensor for storing outputs of current beams
    decoder_input_ids = torch.full(  #making a 2D tensor of shape (original_batch_size * num_beams, 1) with each element equal to "BOS" token_id of the decoder/tokenizer
                        (original_batch_size * num_beams, 1),
                        model.config.decoder_start_token_id,
                        dtype=torch.long,
                        device=device
                        )

    # making a 1D tensor for storing scores of all beams of all sequences
    beam_scores = torch.zeros(original_batch_size * num_beams, dtype=torch.float, device=device)

    #Maintains a list of states for each beam
    constraint_states = [[{"active": True, "satisfied": False, "tokens_left": constraint_info["token_ids"].clone(), "position": constraint_info["position"]} for constraint_info in constraints] for _ in range(original_batch_size * num_beams)] #list for each beam of all input sequences, each beam(row) has a list of constraints states
                                                                                                                             #each constraint state is a dictionary with 3 keys- "active"(meaning it is actively looking to be fullfilled), "satisfied"(meaning it is satisfied), "tokens_left"(which contain a copy of the tokens to be included for that constraint in that beam)
    Start=True ##initial Condition to be checked only once during the first forward pass                                                                                                                           #Active=True means the full constraint is pending or part of the constraint is pending
    #Helper function for adding constraint token
    def find_variable_in_constraints(step, constraints):
        position_list=[]
        for i, constraint_dict in enumerate(constraints):
            if step == constraint_dict["position"] and constraint_dict["active"]==True: #Constraint must be active constraint
                position_list.append(i)  # Return the index of the dictionary whose constraint position is equal to current decoding step
        if position_list: #if their is anything in the position list
            return position_list
        return None     #Else return None
    #Decoding Steps for generating the output
    for step in range(max_length):#each decoding step, adding one token to a beam of decoder_input_ids

        #Adding Constraint
        constraint_num=find_variable_in_constraints(step, constraint_states[0]) #checking for position in first beam since all beams are updated together anyways, it will return constraints whose "position" is equal to the current step
        if constraint_num is not None: #this step is equal to position of some constraint
            #add constraint token to decoder_input_id
            constraint_token_tensor=torch.full((original_batch_size * num_beams, 1), constraint_states[0][constraint_num[0]]["tokens_left"][0],dtype=torch.long, device=device)
            """
            Explaination of above step:
            constraint_states[0] gives us a list of dictionaries containing constraint info for all constraints of the first beam(since all constraints for all beams are updated together we can take any one, it doesnt matter)
            then from the constraint_num list that has a list of indices for all constraints that have first token valid for this step, we choose the first one i.e.constraint_num[0](it wouldnt matter if you choose the second one too since the token for a specific position would be the same as they are all derived from the same reference translation)
            Then we take the first token of that constraint and prepare a tensor to add it to the decoder input_ids
            """
            #add the 1 token of constraint to beams of decoder_input_id
            decoder_input_ids=torch.cat([decoder_input_ids, constraint_token_tensor], dim=1)


            #update constraint_state for beams of the sequence
            for i in range(original_batch_size*num_beams): #for all beams
                for x in range(len(constraint_num)):  #for all constraints with position same as this step
                    constraint_states[i][constraint_num[x]]["position"]+=1 #shift position 1 index forward for the next token
                    constraint_states[i][constraint_num[x]]["tokens_left"]=constraint_states[i][constraint_num[x]]["tokens_left"][1:] #remove the token from the constraint that has been added to the decoder_input_id
                    if len(constraint_states[i][constraint_num[x]]["tokens_left"])==0: #when there are no more tokens left in that constraint
                        constraint_states[i][constraint_num[x]]["active"]=False #change the state of the constraint



        #Generating token by use of logits and Scratch beam search
        else: #this step is not a position of some constraint then we just generate using models logits
            with torch.no_grad():

                # Forward pass
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    decoder_input_ids=decoder_input_ids
                    ) #This will be a dictionary like object containing a lot of things like hidden states,etc, but we are most interested in logits
                """
                Talking about some important Tensor Operations:

                Broadcasting:
                In pytorch, a process called broadcasting happens just before an element wise operation that requires two tensors with same number of dimensions to align in shape.
                Example Adding tensor A of shape(4,6,8) with tensor B of shape(4,1,1) then the scaler values in tensor B along the dimensions with only 1 element are duplicated to match the shape of the tensor A.
                But broadcasting only happens when two tensors have same size of some dimensions and some dimensions of size 1.

                tensor.view(size of dim=0, size of dim=1.....) is used to create a reshaped tensor with the same number of elements as the original tensor.
                It can add new dimensions of a specific size as long as the number of element in the original tensor could be accomodated.
                tensor.view(-1,10) tell us to accomodate 10 elements in dim=10 and infer what number of element to accomodate in dim=0 such that the total number of elements remain the same.

                tensor.unsqueeze() is used to add a singleton dimension(dimension of size 1) at a specified index
                tensor.unsequeeze(1) adds a singleton dimension at index=1

                tensor.squeeze() is used remove a singleton dimension from a specified index

                tensor.expand() is used to basically perform broadcasting on command, you would send in sizes of dimension of tensor which are capable of broadcasting and get a result.
                This means that the size you send in should either be the same as sizes of dimension of original tensor OR in case of singleton dimension in original tensor you could send in a greater number to stretch along that dimension
                tensor.expand(-1, 4) tells to keep dim=0 as is and basically stretch dim=1 to size 4, this is conditioned on the fact that the dimension to be stretched is of size 1 otherwise an error will occur

                tensor.expand_as(target_tensor) is used to perform broadcasting on tensor conditioned on the fact that the size of target_tensor is suitable for broadcasting


                """
                # Creating a 2D tensor Logits
                logits = outputs.logits[:, -1, :] #output.logits is a 3-D tensor of shape(original_batch_size*num_beam, current sequence length in decoder_input_ids, vocab size). where each element is the logit of each token_id of each sequence for each element of the vocabulary
                                                #By slicing we choose the last token_id of each sequence it to get a 2-D tensor of shape(original_batch_size*num_beam, vocab_size) where each element is a logit of last token_id of each sequence element for each element of the vocabulary
                #print(f"Logits at step {step}:\n{logits}")
                # Using Boolean masking to mark finished beams (those that already generated EOS)
                #finished_mask = finished_beams.view(-1, 1).expand(-1, logits.size(-1))  # finished_mask will be a 2D of shape(original_batch_size * num_beams, vocab_size) with values of True/False repeated in each row(representing each beam) upto the vocab_size
                #logits[finished_mask] = -float('inf')  # Making logits of lasts token for all elements of vocabulary for the finished beams negative infinity, By sending in boolean mask tensor of the same shape as the logits tensor the operation is applied to all logits elements that correspond with True in finished mask.
                                                        #Making Logit for all words in vocabulary negative infinity means probability=0 after softmax, and log of that is a big negative value, when that big negative value is combined with beam_score to get next_score it ensures that these beams will now not come in the topk.
                                                        #This ensures that only active beams (those not yet finished) are considered during the top-k selection.

                # Creating 2D tensor log probabilities
                log_probs = F.log_softmax(logits, dim=-1) #performs softmax on all logits normalizing them to probability of 0-1, then takes log of that so the highest probability is turend into the largest number(smallest negative value).
                                                        #dim=-1 takes softmax along the last dimension(2nd dimension with size=vocab_size in this case) which means that normalization happens for each row(last token for each sequence of decoder_input_ids)
                #print(f"Log_probs at step {step}:\n{log_probs}")
                #get top num_beams scores from each beam
                total_top_log_probs, total_top_token_ids=torch.topk(log_probs, num_beams, dim=1) #give 2 tensors of shape(original_batch_size*num_beams, num_beams) where each row contains scores top num_beam scores for that row and the corresponding token_ids in descending
                #print(f"Topk log_probs at step {step}:\n{total_top_log_probs}")
                #print(f"Topk token_ids at step {step}:\n{total_top_token_ids}")

                if Start==True:
                    new_tokens=total_top_token_ids[::num_beams].reshape(-1,1) #using slicing with step value num_beam we take the top 4 tokens of a beam of a sequence and make a tensor containing new tokens for each beam, .reshape is used in place of .view() becauce we used slicing and so tensor is not contiguous in memory hence .view wont work
                    decoder_input_ids=torch.cat([decoder_input_ids[torch.arange(original_batch_size*num_beams)],new_tokens], dim=1) #adding first 4 tokens to different beam of each sequence
                    beam_scores=total_top_log_probs[::num_beams].reshape(-1) #slicing to make tensor of shape(original_batch_size,num_beams) and then we flatten it using .reshape(-1)
                    Start=False
                    #print(f"Decoder_input_ids at step{step}:\n{decoder_input_ids}")
                    #print(f"Beam score at step{step}:\n{beam_scores}")

                    continue

                #calculate total scores for each beams top 4 tokens
                total_top_score=total_top_log_probs+beam_scores.unsqueeze(1)
                #print(f"Total score of each topk token of each beam scores at step {step}:\n{total_top_score}")


                #reshape that
                reshaped_total_top_score=total_top_score.view(original_batch_size, num_beams*num_beams)

                #Getting the best scores for this step in generation
                new_top_scores,new_token_indices=torch.topk(reshaped_total_top_score,num_beams, dim=1) #gives 2 tensors of shape (original_batch_size, num_beam) containing the top num_beam score from all total_scores of one sequence in each row in descending order and their index in that row in total_top_score_reshaped
                #print(f"Total Top scores for new tokens of each sequence scores at step {step}:\n{new_top_scores}")
                #calculating beam indices for the new tokens of each sequence with best score
                beam_indices=new_token_indices//num_beams #elements of a row in this tensor will tell which token originally belonged to which beam for a sequence
                #reshaping beam_indices and scaling indices of elements from row level to global level in the tensor
                base_indices=torch.arange(original_batch_size,device=device).unsqueeze(1) * num_beams
                scaled_beam_indices=beam_indices+base_indices #Tensor of shape shape (original_batch_size, num_beam)
                flattened_scaled_beam_indices=scaled_beam_indices.view(-1) #making a 1D tensor of shape(original_batch_size*num_beam)

                #By Advance Indexing using these scaled indices we will obtain sequences from decoder_input_ids of the respective beam that the indices point to
                prev_decoder_inputs=decoder_input_ids[flattened_scaled_beam_indices]

                #Getting the token_ids of the tokens to be added
                reshaped_top_total_token_ids=total_top_token_ids.view(original_batch_size, num_beams*num_beams)
                """
                Here each row will contain the token ids of the top scoring tokens of all beams of a sequence just like the structure of reshaped_total_top_score
                Since new_token_indices tells the row index of the topk elements of that row, the same indices will also point to the token_ids in reshaped_top_total_token_ids
                So we make a vector of shape (original_batch_size,1) where each element will help us point to a row indices in reshaped_top_total_token_ids
                And then new_token_indices has column indices which help us point to columns of reshaped_top_total_token_ids for that row
                in this case of Advacne indexing because we use two rows and columns we get a new tensor of shape of rows and columns used
                """

                raw_new_tokens = reshaped_top_total_token_ids[torch.arange(original_batch_size,device=device).unsqueeze(1), new_token_indices] #the token_ids for the new tokens in a Tensor of shape(original_batch_size, num_beams)
                new_tokens = raw_new_tokens.view(original_batch_size*num_beams,1) #2D Tensor of shape(original_batch_size*num_beam,1) made to be suited for concatenation to decoder_input_ids
                decoder_input_ids=torch.cat([prev_decoder_inputs, new_tokens], dim=1)

                #Updating the beam scores to be used after this step in generation
                beam_scores=new_top_scores.view(-1)

                #Printing decoder_input_ids to see whats going on
                #print(f"Decoder input ids at step{step}:\n{decoder_input_ids}")

                # Check for early stopping
                if early_stopping:
                    # If all beams for all sequences are finished, stop decoding
                    if new_tokens[0]==tokenizer.eos_token_id: #if the top beam at the time generates the EOS token then break the loop
                        break


    # Reshape and extract the best sequence for each input
    output_sequences = decoder_input_ids.view(original_batch_size, num_beams, -1)
    best_sequences = output_sequences[:, 0, :] #creates 2D tensor of shape(original_batch_size, sequence length)

    return best_sequences


# Requires a translated list(list containing MT hypothesis) of previous iteration and index of current iteration
def constrained_translate(translated_list):
    #Tokenizing translated list

    translated_list_t = tokenizer(
                translated_list,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128
                ).to(device)
    translated_input_ids=translated_list_t["input_ids"] #This will be a 2D tensor of shape(total number of sequences tokenized, sequence length)
    translated_attention_mask=translated_list_t["attention_mask"] #This will be 2D tensor of shape(total number of sequences tokenized, sequence length)

    dataset_t = tokenizer(
                source_reference_list,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128
                ).to(device)
    source_input_ids=dataset_t["input_ids"] #This will be a 2D tensor of shape(total number of sequences tokenized, i.e. length of list passed in tokenizer, sequence length)
    source_attention_mask=dataset_t["attention_mask"] #This will be 2D tensor of shape(total number of sequences tokenized, i.e. length of list passed in tokenizer, sequence length)


    #Tokenizing target(russian) reference list
    reference_list_t = tokenizer(
            target_reference_list,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
            ).to(device)

    reference_input_ids = reference_list_t["input_ids"]  # This will be 2D tensor of shape(total number of sequences tokenized, sequence length)
    reference_attention_mask = reference_list_t["attention_mask"]  # This will be 2D tensor of shape(total number of sequences tokenized, sequence length)

    constrained_translated_list=[]


    # Iterating through each translated sequence
    for i in tqdm(range(len(translated_list))):
        translated_seq_ids = translated_input_ids[i:i+1]  # 2D tensor of shape [1, max seq_length in the tokenized list] containing token_ids for Single sequence from translated batch
        reference_seq_ids = reference_input_ids[i:i+1]  # 2D tensor of shape [1, max seq_length in the tokenized list] containing token_ids for Corresponding reference sequence
        source_seq_ids= source_input_ids[i:i+1] # 2D tensor of shape [1, max seq_length in the tokenized list] containing token_ids for Corresponding reference sequence
        source_seq_attention_mask=source_attention_mask[i:i+1]


        # Find phrases(subsequences) missing from translated_seq to add them as constraints
        constraint, position = find_missing_tokens(translated_seq_ids[0], reference_seq_ids[0])#constraint will be a 1D tensor of token_ids, Sending in values of the one row in translated_seq_ids and reference_seq_ids in find_missing_tokens, so these are 1D tensors(imagine like a list of integers)
                                                                                    #it could also return None in case of no mismatch
        print(f"The reference sequence Token id:\n{reference_seq_ids[0]}")
        print(f"The Hypotheses sequence Token id:\n{translated_seq_ids[0]}")
        print(f"constraint Identified in this iteration at position {position}:\n{constraint}")
        #In case there is no mismatch in translated and reference list just add the sequence from translated_list in the constrained_translated_list, and continue to next sequence
        if constraint==None:
            constrained_translated_list.append(translated_list[i]) #.append will add the string as a single element to constrained_translated_list, if we use .extend here it will treat the string as an iterable and add each of its elements(characters) to the constrained_translated_list
            continue

        constraint_info={"token_ids":constraint, "position":position}
        #Add the constraint to the list of constraints for the current sequence
        GBS_constraints[i].append(constraint_info)


        #Generated translation using Grid_beam_search
        constrained_seq_ids = constrained_beam_search(
            input_ids=source_seq_ids,#sending input_ids of one sequence
            attention_mask=source_seq_attention_mask, #sending attention mask of 1 sequence
            constraints=GBS_constraints[i], #Sending constraints for that one sequence
        )

        print(f"token_ids of the Constrained_Beam_Search output which will be decoded and stored as our constrained translation:\n{constrained_seq_ids}")
        constrained_translation = tokenizer.batch_decode(constrained_seq_ids, skip_special_tokens=True)

        constrained_translated_list.extend(constrained_translation) #we use .extend here because it will add each element of the iterable constrained_translation to constrained_translated_list, constrained_translation could be a list of strings, but in this case is just a list containing one string

    return constrained_translated_list


#Getting the translation of input sequence by deriving constraints for each sequence of target_reference_list subsequences that are missing from model_generated_list
constrained_translation_1=constrained_translate(model_generated_list)  #list of Constrained Translated Sequnces in Target Language

Constraint_Seq_t=tokenizer(
                constrained_translation_1,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128
                ).to(device)
constraint_seq_input_ids=Constraint_Seq_t["input_ids"]
print(f"The token_ids after tokenizing the constrained translation showing the constrained token_ids are still applied:\n{constraint_seq_input_ids}")


#BLEU score for fist iteration
Base_blue_score= sacrebleu.corpus_bleu(model_generated_list, reference_list_BLEU)
print(f"\nBase Bleu score:{Base_blue_score} ")
#Confirm number of elements in hypotheses list is equal to number of sequences in reference list)
if len(constrained_translation_1)==len(reference_list_BLEU[0]):
    constrained_bleu_score1= sacrebleu.corpus_bleu(constrained_translation_1, reference_list_BLEU)
    print(f"\n BLEU score for the constrained translation Sequence:{constrained_bleu_score1}")

else:
    print("Mismatch in number of hypotheses and reference sequences")



  0%|          | 0/1 [00:00<?, ?it/s]

The reference sequence Token id:
tensor([  21, 1903,   41,  222,  779,   70,  222,   38,   21, 4113,   70,  230,
         230,   53,  229,   21,  141,   30,   21,  230,  275,   41,  141,   30,
         275,   71,   44,  374,   70,  971,   41,  275,  426,   21,  260,  222,
          30,  779,  426,  173,  198,   21,  222,   21,  230,  222,  229,  670,
          53,   21,  230,   21,  141,   70,  222,  100,   46,   53,   21,  230,
          41,  141,  557, 3699,   53,  229,   46,   53,   21,   53,   21,  141,
          30,   21, 1209,   60,  260,   30,  275,   21,  222,  100,   44,   70,
         260,   53,  275,  426,   21,   53,  670,   21,  746, 1676, 1755,    0])
The Hypotheses sequence Token id:
tensor([  21, 4113,   70,  230,  230,   53,  229,   21,  141,   30,   21,  670,
          41,   44,  374,   70,  971,  141,   30,  275,   21,  260,  222,   30,
         779,  426,   21,  971,   30,  779,   30,  260,   21,  374,   53, 3699,
          70,   46,   21,  141,   70,  222,  100,   

100%|██████████| 1/1 [00:03<00:00,  3.62s/it]

token_ids of the Constrained_Beam_Search output which will be decoded and stored as our constrained translation:
tensor([[62517,    21,  1903,    41,   222,  4053,    38, 23453,    26,    52,
            44, 20391,  3573,  8471,   916,  7014,  1149,  6170,     7,    26,
            15,   451,  3573,  8271,     0]])
The token_ids after tokenizing the constrained translation showing the constrained token_ids are still applied:
tensor([[  21, 1903,   41,  222,  779,   70,  222,   38,   21, 4113,   70,  230,
          230,   53,  229,   21,  141,   30,   21,  670,   41,   44,  374,   70,
          971,  141,   30,  275,   21,  260,  222,   30,  779,  426,   21,  971,
           30,  779,   30,  260,   21,  374,   53, 3699,   70,   46,   21,  141,
           70,  222,  100,   44,   21,  230,   41,  141,  557, 3699,   53,   48,
           21,   53,   21,  141,   30,   21,  971,   70,  557,   53,  141,   30,
          275,   21,  746, 1676, 1755,    0]])

Base Bleu score:BLEU = 2.81 22.2/12.5